# 01 Riluzole Study Biomarkers - Clean and combine data

diagnostic nocturnal polysomnogram			DNP
portable diagnostic sleep study 			PDSS
nocturnal polysomnogram			NP
split-night polysomnographic report 			SNPR

RIL APE = Riluzole study

In [1]:
# install required packages - commented out so it doesn't install every time
#%conda install -n Lauren openpyxl numpy pandas statsmodels plotnine matplotlib scikit-learn scipy mizani nbconvert pandoc pyreadstat kmodes seaborn

# import required packages
import openpyxl; 
import numpy as np;
import pandas as pd;
import statsmodels as sm;
import statsmodels.formula.api as smf;
import plotnine as p9;
import pickle

data_folder_loc = "//admsyn/Primary/ADM/CustomerStudies/Rockefeller/Riluzole_Biomarkers/"
code_folder_loc = "//admsyn/homes/@DH-ADMDX/0/lauren.koenig-1606/code/Riluzole FDG/"

# FDG
data_FDG_rr_orig from FDG PET Riluzole_Raw Data.xlsx (4/2/24)
data_FDG_rr_pons from Riluzole_Baseline_FDG_Vol_values.xlsx (4/26/24)
data_FDG_rr_para from Riluzole_Baseline_FDG_Vol_values.xlsx (4/26/24)
FDG_pons from RIL_FDG_NormedtoPons_1.xlsx (4/25/24)

data_FDG_randy_rr_cerebellum from Rockefeller_Database__Ver27__090920.xlsx
data_FDG_randy_rr_wbsv from Rockefeller_Database__Ver27__090920.xlsx
data_FDG_randy_rr_pons from Rockefeller_Database__Ver27__090920.xlsx
data_FDG_randy_rr_mean from Rockefeller_Database__Ver27__090920.xlsx
data_FDG_randy_voi from Rockefeller_Database__Ver27__090920.xlsx


data_FDG_rr_para_new from RIL_for_pTau_Correlation_082124.xlsx  (8/21/24)
data_FDG_rr_ras_new from Referenced to RAS-ref1.xlsx (8/21/24)

In original analyses, used data from Riluzole_Baseline_FDG_Vol_values.xlsx and FDG PET Riluzole_Raw Data.xlsx

## original RR

In [2]:
data_FDG_rr_orig_wide = pd.read_excel(data_folder_loc + "original\discrete\FDG PET Riluzole_Raw Data.xlsx", sheet_name = 'FDG SUVRs', skiprows = 4)

In [3]:
data_FDG_rr_orig_wide['Subject_ID'] = data_FDG_rr_orig_wide['Subject_ID'].replace('-o', '', regex = True).replace('APE-','APE-792-',   regex = True)

# some regions have baseline and change listed, but not 6 mo data, so calculate it
data_FDG_rr_orig_wide['Avg_MedOrbFrontal_6m']  = data_FDG_rr_orig_wide['Avg_MedOrbFrontal_DIF'] + data_FDG_rr_orig_wide['Avg_MedOrbFrontal']
data_FDG_rr_orig_wide['Temp-gm_6m']  = data_FDG_rr_orig_wide['Temp-gm_DIF'] + data_FDG_rr_orig_wide['Temp-gm_BL']
data_FDG_rr_orig_wide['Temp_6m']  = data_FDG_rr_orig_wide['Temp_DIF'] + data_FDG_rr_orig_wide['Temp_BL']
data_FDG_rr_orig_wide['AC-gm_6m']  = data_FDG_rr_orig_wide['AC-gm_DIF'] + data_FDG_rr_orig_wide['AC-gm_BL']

# separate out baseline and 6 mo measurements in order to reshape data into long format
data_FDG_rr_orig_bl = data_FDG_rr_orig_wide[['Subject_ID', 
       'L Hip_BL', 'R Hip_BL', 'Avg HIP_BL', 
       'Temp_BL', 'Temp-gm_BL', 
       'MTL-gm_BL', 
       'PostCing-gm_BL', 
       'Precun-gm_BL',
       'Avg PCC_BL', 
       'Par-gm_BL', 
       'AC-gm_BL', 
       'FRONTAL-gm_BL',
       'Avg_MedOrbFrontal']]

data_FDG_rr_orig_bl['timepoint'] = 'base'
data_FDG_rr_orig_bl.columns = data_FDG_rr_orig_bl.columns.str.replace('_BL', '')

data_FDG_rr_orig_6mo = data_FDG_rr_orig_wide[['Subject_ID', 
       'L Hip_6m', 'R Hip_6m', 'Avg HIP_6m', 
       'Temp_6m', 'Temp-gm_6m',
       'MTL-gm_6m',
       #'Temp iso_6m', # extra
       'PostCing-gm_6m', 
       'Precun-gm_6m', 
       'Avg PCC_6m',
       'Par-gm_6m', 
       'AC-gm_6m',
       'FRONTAL-gm_6m', 
       'Avg_MedOrbFrontal_6m']]

data_FDG_rr_orig_6mo.columns = data_FDG_rr_orig_6mo.columns.str.replace('_6m', '')
data_FDG_rr_orig_6mo['timepoint'] = 'end'


# reshape data into long format
data_FDG_rr_orig = pd.merge(data_FDG_rr_orig_bl, data_FDG_rr_orig_6mo, how = 'outer' )

data_FDG_rr_orig = data_FDG_rr_orig.rename(columns = {'L Hip':'L_Hip', 'R Hip':'R_Hip', 'Avg HIP':'Avg_Hip'})

data_FDG_rr_orig['Subject_Label'] = data_FDG_rr_orig['Subject_ID'].str[0:-4]
data_FDG_rr_orig['Subject_Number'] = data_FDG_rr_orig['Subject_ID'].str[-3:].astype('int')
data_FDG_rr_orig['ID_ADM'] = data_FDG_rr_orig['Subject_Label'] + '_' + data_FDG_rr_orig['Subject_Number'].astype('str')


C:\Users\Lauren.Koenig\AppData\Local\Temp\8\ipykernel_38976\2662532944.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Lauren.Koenig\AppData\Local\Temp\8\ipykernel_38976\2662532944.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## pons and PARA RR

In [4]:
data_pons_freesurfer_ril = pd.read_excel(data_folder_loc + 'original\discrete\Riluzole_Baseline_FDG_Vol_values.xlsx', skiprows = 1)

data_pons_freesurfer_ril['Subject_ID'] = data_pons_freesurfer_ril['Subject_ID'].replace('APE-','APE-792-',   regex = True)
data_pons_freesurfer_ril['Subject_Label'] = data_pons_freesurfer_ril['Subject_ID'].str[0:-4]
data_pons_freesurfer_ril['Subject_Number'] = data_pons_freesurfer_ril['Subject_ID'].str[-3:].astype('int')
data_pons_freesurfer_ril['ID_ADM'] = data_pons_freesurfer_ril['Subject_Label'] + '_' + data_FDG_rr_orig['Subject_Number'].astype('str')
data_pons_freesurfer_ril['timepoint'] = 'base'


data_freesurfer_ril_z = data_pons_freesurfer_ril[['Subject_ID','timepoint', 'Subject_Number', 'Subject_Label', 'ID_ADM', 'VOL_Precun_Lz', 'VOL_Precun_Rz',
       'VOL_InfPar_Lz', 'VOL_InfPar_Rz', 'VOL_Hip_Lz', 'VOL_Hip_Rz',
       'VOL_TotalGrayz', 'VOL_Inf_Mid_Fus_Temp_Lz', 'VOL_Inf_Mid_Fus_Temp_Rz',
       'VOL__MidFront_Lz', 'VOL_MidFront_Rz']]

data_FDG_rr_pons = data_pons_freesurfer_ril[['Subject_ID','timepoint', 'Subject_Number', 'Subject_Label', 'ID_ADM', 'Temp-gm_ponsref', 'MTL-gm_ponsref',
       'PostCing-gm_ponsref', 'Par-gm_ponsref', 'Precun-gm_ponsref',
       'AC-gm_ponsref']]


data_FDG_rr_para = data_pons_freesurfer_ril[['Subject_ID','timepoint', 'Subject_Number', 'Subject_Label', 'ID_ADM', 'Temp-gm_refpara2', 'MTL-gm_refpara2',
       'PostCing-gm_refpara2', 'Par-gm_refpara2', 'Precun-gm_refpara2',
       'AC-gm_refpara2', 'Avg PCC_refpara2']]

data_FDG_rr_pons.columns = data_FDG_rr_pons.columns.str.replace('_ponsref', '')
data_FDG_rr_para.columns = data_FDG_rr_para.columns.str.replace('_refpara2', '')

## pons RR

In [5]:
FDG_pons = pd.read_excel(data_folder_loc + 'original\discrete\RIL_FDG_NormedtoPons_1.xlsx', skiprows = 1)

In [6]:
FDG_pons

# fix column names so consistent
FDG_pons = FDG_pons.rename(columns = {'Visit':'timepoint'})
FDG_pons['timepoint'] = FDG_pons['timepoint'].replace({'BL':'base', 'm6':'end'})
FDG_pons.columns = FDG_pons.columns.str.replace(' |-', '_', regex = True)

# add 792 to ID to make consistent with other sheet's IDs
FDG_pons['Subject_ID'] = FDG_pons['Subject_ID'].replace('APE-','APE-792-',   regex = True)

FDG_pons = FDG_pons.dropna(subset = ['Subject_ID'])

FDG_pons['Subject_Label'] = FDG_pons['Subject_ID'].str.split('-', expand = True)[0]
FDG_pons['Subject_Number'] = FDG_pons['Subject_ID'].str.split('-', expand = True)[1].astype('int')
# add 792 to ID to make consistent with other sheet's IDs
FDG_pons['Subject_ID'] = FDG_pons['Subject_ID'].replace('APE-','APE-792-',   regex = True)
FDG_pons['Subject_Label'] = FDG_pons['Subject_Label'].replace('APE','APE-792',   regex = True)

FDG_pons['ID_ADM'] = FDG_pons['Subject_Label'] + '_' + data_FDG_rr_orig['Subject_Number'].astype('str')


## Randy's db

In [7]:
randy_db = pd.read_excel(data_folder_loc + "original/discrete/Rockefeller_Database__Ver27__090920.xlsx", sheet_name='RKF Database', skiprows=13)

In [8]:
temp = pd.read_excel(data_folder_loc + "original/discrete/Rockefeller_Database__Ver27__090920.xlsx", sheet_name='RKF Database')
temp = temp[0:13]
temp = temp.dropna(how = 'all', axis = 0).reset_index(drop = True)
temp = temp.T
temp = temp.reset_index()
temp[8] = randy_db.columns
temp.loc[temp['index'].str.contains('Unnamed:')==True, 'index'] = np.nan
temp = temp.drop(columns = [0,1,2,3,4,5,6])

temp['section'] = np.nan

temp.loc[temp[7].isin(['(Norm\'d to Pons)']),'section'] = "FDG Pons RR"
temp.loc[temp[7].isin(['(Norm\'d to WBsV)']),'section'] = "FDG WBsV RR"
temp.loc[temp[7].isin(['(Norm\'d to Cerebellum)']),'section'] = "FDG Cerebellum RR"
temp.loc[temp[7].isin(['VOI Number']),'section'] = "FDG VOI Number"
temp.loc[temp[7].isin(['MEAN']),'section'] = "FDG MEAN"


temp.loc[temp[8].isin(['GM', 'WM', 'CSF', 'Total']),'section'] = "SPM12"

idx = temp[7].isin(['FreeSurfer v6.0 -->'])
idx[idx.index>idx[idx==True].index.values[0]] = True # fill in those after the start as true
temp.loc[idx, 'section'] = "FS 6.0"

In [9]:
randy_db = randy_db.replace('pending', np.nan)
randy_db = randy_db[randy_db.columns[~randy_db.isna().all(axis = 0)]]

fs_nonvol_cols = randy_db.columns[randy_db.columns.str.contains('_area_|_meancurv_|_thickavg_')].to_list()
randy_db = randy_db.drop(columns = fs_nonvol_cols)

fs_A2009s_cols = randy_db.columns[randy_db.columns.str.contains('A2009s')].to_list()
randy_db = randy_db.drop(columns = fs_A2009s_cols)

fs_BA_cols = randy_db.columns[randy_db.columns.str.contains('BAex')].to_list()
randy_db = randy_db.drop(columns = fs_BA_cols)

fs_WMparc_cols = randy_db.columns[randy_db.columns.str.contains('WMparc')].to_list()
randy_db = randy_db.drop(columns = fs_WMparc_cols)

fs_aparc_cols = randy_db.columns[randy_db.columns.str.contains('AparcSt') & ~randy_db.columns.isin(['AparcSt_vol_L_SSTSBANK',
 'AparcSt_vol_L_FRNPOLE',
 'AparcSt_vol_L_TEMPPOLE',
 'AparcSt_vol_R_SSTSBANK',
 'AparcSt_vol_R_FRNPOLE',
 'AparcSt_vol_R_TEMPPOLE'])].to_list()
randy_db = randy_db.drop(columns = fs_aparc_cols)

# drop useless columns
randy_db = randy_db.drop(columns = ['INDEX-2', 'ProcDate','FDG multiframe?', 'INDEX-1', 'Completed Protocol? (FDG)', 
'Completed Protocol? (MRI)', 'Visit (ALPHA) FDG', 'Visit (ALPHA)  MRI',  'Image source', 'Unnamed: 15', 'Visit (ADM)',
'CV1:', 'CV2:', 'CV3:', 'CV4:', 'CV5:', 'P_NL:', 'P_MA:', 'P_AD:', 'P_SAD:', 'P_FTD:', 'P_LBD:', 'P_SD:', 'PredictedClass:',
'CV1:.1', 'P_1.1:', 'P_1.2:', 'P_2.1:', 'P_2.2:', 'P_4.1:', 'P_6.1:', 'P_6.2:', 'P_7.1:', 'P_8.1:', 'P_8.3:',
 'PredictedClass:.1', 'Unnamed: 17', 'x',  '(recovered, pending procs)', 'Image data format', 'FileName:.1', 
 'Unnamed: 218','Comment(s) -- MRIs as received','YYYYMMDD', 'Comment(s)'
])

# drop empty rows / not subject data rows
randy_db = randy_db[~(randy_db['Subject'].isna())]

randy_db = randy_db.rename(columns={'Subject':"Subject_ID", 'AGE':'age', 'Gender':'sex', 'FDG Visit':'timepoint'})
randy_db['timepoint'] = randy_db['timepoint'].replace({'Visit-1 (FDG)':'base', 'Visit-1.5 (MRI)':'mid', 'Visit-2 (FDG)':'end',
       'Visit-1 only':'base'})
randy_db['Subject_ID'] = randy_db['Subject_ID'].str.replace('RKF-APE-', 'APE-792-', regex = True).str.replace('RKF-RIL-', 'RIL-', regex = True)
randy_db['Subject_Label'] = randy_db['Subject_ID'].str[0:-4]
randy_db['Subject_Number'] = randy_db['Subject_ID'].str[-3:].astype('int')
randy_db['ID_ADM'] = randy_db['Subject_Label'] + '_' + randy_db['Subject_Number'].astype('str')


FS_cols = randy_db.columns[randy_db.columns.isin(temp.loc[temp['section'].isin(['FS 6.0', 'SPM12']), 8].to_list())].to_list()
FDG_cols = randy_db.columns[randy_db.columns.isin(temp.loc[temp['section'].str.contains('FDG')==True, 8].to_list())].to_list()

# separate out clinical data
data_clin_randy = randy_db[['timepoint', 'ID_ADM', 'Subject_Label', 'Subject_Number', 'age', 'sex']]

# separate out volumetric data
data_freesurfer_ril_randy = randy_db[['timepoint', 'ID_ADM', 'Subject_Label', 'Subject_Number', 'MRI Visit Date', '7-Character machine UID',
       'MRI ReName', 'SPM12 Source Mat file'] + FS_cols]

### Cerebellum RR: Not including in merge

In [10]:
# Cerebellum RR: (data_FDG_rr_orig's closest counterpart)
data_FDG_randy_rr_cerebellum = randy_db[['ID_ADM', 'Subject_Label', 'Subject_Number', 'timepoint', 'FDG Visit Date (from DICOM)', 'FileName:', 'LPT Source'] + randy_db.columns[randy_db.columns.isin(temp.loc[temp['section'].isin(['FDG Cerebellum RR']), 8].to_list())].to_list()].sort_values('ID_ADM')
data_FDG_randy_rr_cerebellum = data_FDG_randy_rr_cerebellum[~data_FDG_randy_rr_cerebellum.drop(columns = ['ID_ADM', 'Subject_Label', 'Subject_Number', 'timepoint', 'FDG Visit Date (from DICOM)', 'FileName:', 'LPT Source']).isna().all(axis = 1)]


### WBsV RR: Not including in merge

In [11]:
#  WBsV RR: (data_FDG_rr_para's closest counterpart)
data_FDG_randy_rr_wbsv = randy_db[['ID_ADM', 'Subject_Label', 'Subject_Number', 'timepoint', 'FDG Visit Date (from DICOM)', 'FileName:', 'LPT Source'] + randy_db.columns[randy_db.columns.isin(temp.loc[temp['section'].isin(['FDG WBsV RR']), 8].to_list())].to_list()].sort_values('ID_ADM')
data_FDG_randy_rr_wbsv = data_FDG_randy_rr_wbsv[~data_FDG_randy_rr_wbsv.drop(columns = ['ID_ADM', 'Subject_Label', 'Subject_Number', 'timepoint', 'FDG Visit Date (from DICOM)', 'FileName:', 'LPT Source']).isna().all(axis = 1)]


### Pons RR: Not including in merge

In [12]:
# Pons RR: (data_FDG_rr_pons's and FDG_pons' closest counterpart)
data_FDG_randy_rr_pons = randy_db[['ID_ADM', 'Subject_Label', 'Subject_Number', 'timepoint', 'FDG Visit Date (from DICOM)', 'FileName:', 'LPT Source'] + randy_db.columns[randy_db.columns.isin(temp.loc[temp['section'].isin(['FDG Pons RR']), 8].to_list())].to_list()].sort_values('ID_ADM')
data_FDG_randy_rr_pons = data_FDG_randy_rr_pons[~data_FDG_randy_rr_pons.drop(columns = ['ID_ADM', 'Subject_Label', 'Subject_Number', 'timepoint', 'FDG Visit Date (from DICOM)', 'FileName:', 'LPT Source']).isna().all(axis = 1)]


### Mean RR: Not including in merge

In [13]:
# Mean RR
data_FDG_randy_rr_mean = randy_db[['ID_ADM', 'Subject_Label', 'Subject_Number', 'timepoint', 'FDG Visit Date (from DICOM)', 'FileName:', 'LPT Source'] + randy_db.columns[randy_db.columns.isin(temp.loc[temp['section'].isin(['FDG MEAN']) , 8].to_list())].to_list()].sort_values('ID_ADM')
data_FDG_randy_rr_mean = data_FDG_randy_rr_mean[~data_FDG_randy_rr_mean.drop(columns = ['ID_ADM', 'Subject_Label', 'Subject_Number', 'timepoint', 'FDG Visit Date (from DICOM)', 'FileName:', 'LPT Source']).isna().all(axis = 1)]

### VOI: Not including in merge

In [14]:
# VOI
data_FDG_randy_voi = randy_db[['ID_ADM', 'Subject_Label', 'Subject_Number', 'timepoint', 'FDG Visit Date (from DICOM)', 'FileName:', 'LPT Source'] + randy_db.columns[randy_db.columns.isin(temp.loc[temp['section'].isin(['FDG VOI Number']), 8].to_list())].to_list()].sort_values('ID_ADM')
data_FDG_randy_voi = data_FDG_randy_voi[~data_FDG_randy_voi.drop(columns = ['ID_ADM', 'Subject_Label', 'Subject_Number', 'timepoint', 'FDG Visit Date (from DICOM)', 'FileName:', 'LPT Source']).isna().all(axis = 1)]

## Paracentral RR with added subjects

In [15]:
data_FDG_rr_para_new = pd.read_excel(data_folder_loc + "processed/RIL_for_pTau_Correlation_082124.xlsx", skiprows=2)

In [16]:
data_FDG_rr_para_new = data_FDG_rr_para_new.rename(columns = {'Unnamed: 1':'Subject_ID', 'TEMP-gm':'Temp-gm'}).drop(columns = ['Unnamed: 0'])

data_FDG_rr_para_new['Subject_ID'] = data_FDG_rr_para_new['Subject_ID'].str.replace('RKF-APE-', 'APE-792-', regex = True).str.replace('RKF-RIL-', 'RIL-', regex = True).str.replace('APE-', 'APE-792-', regex = True).str.replace('APE-792-792-', 'APE-792-', regex = True)
data_FDG_rr_para_new['Subject_Label'] = data_FDG_rr_para_new['Subject_ID'].str[0:-4]
data_FDG_rr_para_new['Subject_Number'] = data_FDG_rr_para_new['Subject_ID'].str[-3:].astype('int')

data_FDG_rr_para_new['ID_ADM'] = data_FDG_rr_para_new['Subject_Label'] + '_' + data_FDG_rr_para_new['Subject_Number'].astype('str')

data_FDG_rr_para_new['timepoint'] = 'base'



## RAS RR with added subjects

In [17]:
data_FDG_rr_ras_new = pd.read_excel(data_folder_loc + "processed/Referenced to RAS-ref1.xlsx", skiprows=1)

In [18]:
data_FDG_rr_ras_new = data_FDG_rr_ras_new.rename(columns = {'Unnamed: 0':'Subject_ID', 'TEMP-gm':'Temp-gm'})

data_FDG_rr_ras_new['Subject_ID'] = data_FDG_rr_ras_new['Subject_ID'].str.replace('RKF-APE-', 'APE-792-', regex = True).str.replace('RKF-RIL-', 'RIL-', regex = True).str.replace('APE-', 'APE-792-', regex = True).str.replace('APE-792-792-', 'APE-792-', regex = True)
data_FDG_rr_ras_new['Subject_Label'] = data_FDG_rr_ras_new['Subject_ID'].str[0:-4]
data_FDG_rr_ras_new['Subject_Number'] = data_FDG_rr_ras_new['Subject_ID'].str[-3:].astype('int')
data_FDG_rr_ras_new['ID_ADM'] = data_FDG_rr_ras_new['Subject_Label'] + '_' + data_FDG_rr_ras_new['Subject_Number'].astype('str')

data_FDG_rr_ras_new['timepoint'] = 'base'

data_FDG_rr_ras_new['ID_ADM']

0     APE-792_18
1     APE-792_26
2          RIL_1
3         RIL_18
4         RIL_19
5         RIL_25
6         RIL_27
7         RIL_28
8      APE-792_1
9     APE-792_10
10    APE-792_11
11    APE-792_27
12    APE-792_53
13    APE-792_58
14         RIL_2
15         RIL_3
16         RIL_9
17        RIL_10
18        RIL_20
19    APE-792_34
20    APE-792_13
21    APE-792_20
22    APE-792_33
23    APE-792_36
24    APE-792_46
25    APE-792_47
26    APE-792_56
27    APE-792_66
28         RIL_7
29        RIL_11
30        RIL_12
31        RIL_14
32        RIL_22
33        RIL_29
34        RIL_31
35     APE-792_5
36     APE-792_7
37    APE-792_16
38    APE-792_50
39    APE-792_62
40         RIL_8
41        RIL_15
42    APE-792_23
43    APE-792_35
44    APE-792_39
45    APE-792_65
46    APE-792_71
47        RIL_17
48        RIL_21
49        RIL_24
50     APE-792_9
51    APE-792_22
52        RIL_13
Name: ID_ADM, dtype: object

# Plasma
Done: data_plasma_all

In [19]:
data_plasma_pt217 = pd.read_excel(data_folder_loc + "original\discrete\ID000854_Results_Study_Ab40_42_NFL_GFAp_pTau181_231_217_HH.xlsx", sheet_name = 'p-tau217 results')
data_plasma_n4pe = pd.read_excel(data_folder_loc + "original\discrete\ID000854_Results_Study_Ab40_42_NFL_GFAp_pTau181_231_217_HH.xlsx", sheet_name = 'N4PE, p-t181  results')
data_plasma_pt231 = pd.read_excel(data_folder_loc + "original\discrete\ID000854_Results_Study_Ab40_42_NFL_GFAp_pTau181_231_217_HH.xlsx", sheet_name = 'p-tau231 results')

## pTau-217 data

In [20]:
# separate out the Subject_ID and timepoint from the Sample ID
data_plasma_pt217['Subject_ID'] = data_plasma_pt217['Sample ID'].str.split(' ', expand = True)[0]
data_plasma_pt217['timepoint'] = data_plasma_pt217['Sample ID'].str.split(' ', expand = True)[1]
data_plasma_pt217['timepoint'] = data_plasma_pt217['timepoint'].replace({'second':'mid'}, regex=True)

# separate out the Subject_ID's prefix and #
data_plasma_pt217['Subject_Label'] = data_plasma_pt217['Subject_ID'].str.replace('\d+', '', regex = True).str.replace('-', '', regex = True).str.replace('AAANDS', 'AANDS', regex = True)
data_plasma_pt217.loc[data_plasma_pt217['Subject_Label'].isin(['APE']),'Subject_Label' ] = data_plasma_pt217.loc[data_plasma_pt217['Subject_Label'].isin(['APE']),'Subject_ID' ].str[:7] # the 2 APEs are different
data_plasma_pt217['Subject_Number'] = data_plasma_pt217['Subject_ID'].replace('APE-792-|APE-767-','APE-',   regex = True).str.replace('[A-Z]+', '', regex = True).str.replace('-', '', regex = True).astype('int')

# recombine prefix and label into a consistent ID
data_plasma_pt217['ID_ADM'] = data_plasma_pt217['Subject_Label'] + '_' + data_plasma_pt217['Subject_Number'].astype('str')

# fix so matches other 2
data_plasma_pt217['Subject_ID'] = data_plasma_pt217['Subject_ID'].replace({'AANDS00035':'AANDS0035', 'AANDS00042':'AANDS0042', 'AAANDS0045':'AANDS0045', 'AANDS00041':'AANDS0041'})

# look at # of subjects
print('\nptau217', data_plasma_pt217['Subject_Label'].isna().value_counts())
print('\nptau217', data_plasma_pt217['Subject_Label'].value_counts())
print('\nptau217', data_plasma_pt217.drop_duplicates(subset = 'ID_ADM')['Subject_Label'].value_counts())


ptau217 False    183
Name: Subject_Label, dtype: int64

ptau217 APE-792    43
RIL        40
CCH        35
APE-767    35
AANDS      20
ADRC        9
BC          1
Name: Subject_Label, dtype: int64

ptau217 CCH        35
APE-767    33
APE-792    27
RIL        23
AANDS      20
ADRC        9
BC          1
Name: Subject_Label, dtype: int64


In [21]:
# fix the timepoint naming
data_plasma_pt217.loc[data_plasma_pt217['Sample ID'].isin(['APE-792-50 10-07-2015', 'APE-792-56 05-25-2016', 'APE-792-62 01-27-2017', 'APE-792-66 05-10-2017', 'APE-792-022 02-04-2015', 
'APE-792-039 06-24-15', 'APE-792-53 11-06-2015', 'APE-792-60 11-26-2016', 'APE-792-009 06-07-2014']), 'timepoint'] = 'base'
data_plasma_pt217.loc[data_plasma_pt217['Sample ID'].isin(['RIL029', 'RIL031', 'RIL-022 12-05-2019', 'APE-792-62 10-13-2017', 'APE-792-66 02-28-2018']), 'timepoint'] = 'end'
data_plasma_pt217.loc[data_plasma_pt217['Sample ID'].isin(['RIL015', 'RIL028', 'APE-792-50 01-27-2016', 'APE-792-56 09-14-2016', 'APE-792-58', 'APE-792-60 11-26-2016', 'APE-792-65 09-20-2017']), 'timepoint'] = 'mid'

# non-trial ppts don't have timepoints
data_plasma_pt217.loc[~data_plasma_pt217['Subject_Label'].isin(['APE-792', 'RIL']), 'timepoint'] = 'not part of trial' 

# for the 2 controls with 2 points, drop to avoid confusion
data_plasma_pt217 =data_plasma_pt217[~data_plasma_pt217['Sample ID'].isin(['APE-767-37 04-29-2015', 'APE-767-33 01-29-2015'])] # keep 'APE-767-37 03-20-2015',  APE-767-33 04-08-2015

dup_ID = data_plasma_pt217.loc[data_plasma_pt217['ID_ADM'].duplicated(), 'ID_ADM']
dup_ID2 = data_plasma_pt217.loc[data_plasma_pt217['ID_ADM'].isin(dup_ID) & ~data_plasma_pt217['timepoint'].isin(['end', 'base', 'mid']), 'ID_ADM']

data_plasma_pt217.loc[data_plasma_pt217['ID_ADM'].isin(dup_ID2), ['Sample ID', 'ID_ADM', 'timepoint', 'pTau217 [ALZpath Ptau217 Quanterix] (pg/mL)']].sort_values(['ID_ADM', 'timepoint'])

,Sample ID,ID_ADM,timepoint,pTau217 [ALZpath Ptau217 Quanterix] (pg/mL)


## pTau-231 data

In [22]:
# separate out the Subject_ID and timepoint from the Sample ID
data_plasma_pt231['Subject_ID'] = data_plasma_pt231['Sample ID'].str.split(' ', expand = True)[0]
data_plasma_pt231['timepoint'] = data_plasma_pt231['Sample ID'].str.split(' ', expand = True)[1]
data_plasma_pt231['timepoint'] = data_plasma_pt231['timepoint'].replace({'second':'mid'}, regex=True)

# separate out the Subject_ID's prefix and #
data_plasma_pt231['Subject_Label'] = data_plasma_pt231['Subject_ID'].str.replace('\d+', '', regex = True).str.replace('-', '', regex = True).str.replace('AAANDS', 'AANDS', regex = True)
data_plasma_pt231.loc[data_plasma_pt231['Subject_Label'].isin(['APE']),'Subject_Label' ] = data_plasma_pt231.loc[data_plasma_pt231['Subject_Label'].isin(['APE']),'Subject_ID' ].str[:7] # the 2 APEs are different
data_plasma_pt231['Subject_Number'] =  data_plasma_pt231['Subject_ID'].replace('APE-792-|APE-767-','APE-',   regex = True).str.replace('[A-Z]+', '', regex = True).str.replace('-', '', regex = True).astype('int')

# recombine prefix and label into a consistent ID
data_plasma_pt231['ID_ADM'] = data_plasma_pt231['Subject_Label'] + '_' + data_plasma_pt231['Subject_Number'].astype('str')

# look at # of subjects
print('\nptau231', data_plasma_pt231['Subject_Label'].isna().value_counts())
print('\nptau231', data_plasma_pt231['Subject_Label'].value_counts())
print('\nptau231', data_plasma_pt231.drop_duplicates(subset = 'ID_ADM')['Subject_Label'].value_counts())


ptau231 False    185
Name: Subject_Label, dtype: int64

ptau231 APE-792    43
RIL        41
APE-767    36
CCH        35
AANDS      20
ADRC        9
BC          1
Name: Subject_Label, dtype: int64

ptau231 CCH        35
APE-767    34
APE-792    27
RIL        23
AANDS      20
ADRC        9
BC          1
Name: Subject_Label, dtype: int64


In [23]:
# fix the timepoint naming
data_plasma_pt231.loc[data_plasma_pt231['Sample ID'].isin(['RIL-010 12-20-2018', 'RIL-011 12-17-2018', 'RIL-012 01-15-2019', 'RIL-014 01-08-2019', 'RIL-015 04-23-2019', 'RIL-018 03-19.2019', 
'RIL-019 05-07.2019', 'RIL-002 06-19-2018', 'RIL-027 06-25-2019', 'RIL-028 03-09-2019', 'RIL-029 09-10-2019', 'RIL-003 07-17-2018', 'RIL-007 09-14-2018', 'RIL-008 10-26-2018', 
'RIL-009 10-22-2018', 'RIL-031 10-30-2019', 'RIL-001 06-07-2018', 'RIL-017 03-18-2019', 'RIL-024 08-01-2019', 'RIL-025 06-21-2019', 'RIL-005 09-25-2018', 'APE-792-001 11-04-2013',
 'APE-792-010 04-18-2014', 'APE-792-013 06-12-2014', 'APE-792-016 09-24-2014', 'APE-792-020 11-13-14', 'APE-792-022 02-04-2015', 'APE-792-026 03-17-15', 'APE-792-027 03-18-2015', 
 'APE-792-033 04-08-2015',  'APE-792-034 09-30-2015', 'APE-792-036 04-15-2015', 'APE-792-039 06-2420-15', 'APE-792-046 08-05-2015', 'APE-792-005 05-14-2014',
  'APE-792-007 03-04-2014', 'APE-792-009 06-27-2014']), 'timepoint'] = 'base'

data_plasma_pt231.loc[data_plasma_pt231['Sample ID'].isin(['RIL-010 06-27-2019', 'RIL-011 06-19-2019', 'RIL-012 07-10-2019', 'RIL-014 06-13-2019', 'RIL-015 11-05-2019', 'RIL-018 09-13-2019',
'RIL-019 11-06-2019', 'RIL-002 01-10-2019', 'RIL-027 12-12-2019', 'RIL-028 03-13-2020', 'RIL-003 03-11-2019', 'RIL-007 03-13-2019', 'RIL-008 05-15-2019', 'RIL-009 04-18-2019',
 'RIL029', 'RIL031', 'RIL-020 11-11-2019', 'RIL-022 12-05-2019', 'APE-792-001 06-27-2014', 'APE-792-034 03-09-2016', 'APE-792-005 11-10-2014']), 'timepoint'] = 'end'

data_plasma_pt231.loc[data_plasma_pt231['Sample ID'].isin(['RIL015', 'RIL028', 'APE-792-010 07-23-2014', 'APE-792-011 07-22-2014', 'APE-792-013 09-05-2014', 'APE-792-016 01-06-2015',
 'APE-792-018 01-13-2015', 'APE-792-020 01-16-2015', 'APE-792-026 06-28-2015', 'APE-792-027 05-21-2015', 'APE-792-033 05-27-2013', 'APE-792-046 01-19-2016', 
 'APE-792-047 11-03-2015', 'APE-792-007 06-23-2014']), 'timepoint'] = 'mid'

# non-trial sleep controls had only 1 timepoint
data_plasma_pt231.loc[~data_plasma_pt231['Subject_Label'].isin(['APE-792', 'RIL']), 'timepoint'] = 'not part of trial' 

# for the 2 controls with 2 points, drop to avoid confusion
data_plasma_pt231 = data_plasma_pt231[~data_plasma_pt231['Sample ID'].isin(['APE-767-37 04-29-2015', 'APE-767-33 2015-01-29'])] # keep 'APE-767-37 03-20-2015',  APE-767-33 04-08-2015

dup_ID = data_plasma_pt231.loc[data_plasma_pt231['ID_ADM'].duplicated(), 'ID_ADM']
dup_ID2 = data_plasma_pt231.loc[data_plasma_pt231['ID_ADM'].isin(dup_ID) & ~data_plasma_pt231['timepoint'].isin(['end', 'base', 'mid']), 'ID_ADM']

data_plasma_pt231.loc[data_plasma_pt231['ID_ADM'].isin(dup_ID2) , 
['Sample ID', 'ID_ADM', 'timepoint', 'pTau231-3 [ptau181 inhouse] (pg/mL)']].sort_values(['ID_ADM', 'timepoint'])

,Sample ID,ID_ADM,timepoint,pTau231-3 [ptau181 inhouse] (pg/mL)


## data_plasma_n4pe data

In [24]:
# separate out the Subject_ID and timepoint from the Sample ID
data_plasma_n4pe['Subject_ID'] = data_plasma_n4pe['Sample ID'].str.split(' ', expand = True)[0]
data_plasma_n4pe['timepoint'] = data_plasma_n4pe['Sample ID'].str.split(' ', expand = True)[1]
data_plasma_n4pe['timepoint'] = data_plasma_n4pe['timepoint'].replace({'second':'mid'}, regex=True)

# separate out the Subject_ID's prefix and #
data_plasma_n4pe['Subject_Label'] = data_plasma_n4pe['Subject_ID'].str.replace('\d+', '', regex = True).str.replace('-', '', regex = True).str.replace('AAANDS', 'AANDS', regex = True)
data_plasma_n4pe.loc[data_plasma_n4pe['Subject_Label'].isin(['APE']),'Subject_Label' ] = data_plasma_n4pe.loc[data_plasma_n4pe['Subject_Label'].isin(['APE']),'Subject_ID' ].str[:7] # the 2 APEs are different
data_plasma_n4pe['Subject_Number'] =  data_plasma_n4pe['Subject_ID'].replace('APE-792-|APE-767-','APE-',   regex = True).str.replace('[A-Z]+', '', regex = True).str.replace('-', '', regex = True).astype('int')

# recombine prefix and label into a consistent ID
data_plasma_n4pe['ID_ADM'] = data_plasma_n4pe['Subject_Label'] + '_' + data_plasma_n4pe['Subject_Number'].astype('str')

# look at # of subjects
print('\nn4pe', data_plasma_n4pe['Subject_Label'].isna().value_counts())
print('\nn4pe', data_plasma_n4pe['Subject_Label'].value_counts())
print('\nn4pe', data_plasma_n4pe.drop_duplicates(subset = 'ID_ADM')['Subject_Label'].value_counts())


n4pe False    188
Name: Subject_Label, dtype: int64

n4pe APE-792    46
RIL        41
APE-767    36
CCH        35
AANDS      20
ADRC        9
BC          1
Name: Subject_Label, dtype: int64

n4pe CCH        35
APE-767    34
APE-792    27
RIL        23
AANDS      20
ADRC        9
BC          1
Name: Subject_Label, dtype: int64


In [25]:
# fix the timepoint naming
data_plasma_n4pe.loc[data_plasma_n4pe['Sample ID'].isin(['APE-792-022 02-04-2015', 'APE-792-039 06-24-15', 'APE-792-009 06-07-2014']), 'timepoint'] = 'base'
data_plasma_n4pe.loc[data_plasma_n4pe['Sample ID'].isin(['RIL029', 'RIL031']), 'timepoint'] = 'end'
data_plasma_n4pe.loc[data_plasma_n4pe['Sample ID'].isin(['RIL015', 'RIL028']), 'timepoint'] = 'mid'

# non-trial ppts don't have timepoints
data_plasma_n4pe.loc[~data_plasma_n4pe['Subject_Label'].isin(['APE-792', 'RIL']), 'timepoint'] = 'not part of trial' 

# for the 2 controls with 2 points, drop to avoid confusion
data_plasma_n4pe = data_plasma_n4pe[~data_plasma_n4pe['Sample ID'].isin(['APE-767-37 04-29-2015', 'APE-767-33 2015-01-29'])] # keep 'APE-767-37 03-20-2015',  APE-767-33 04-08-2015

dup_ID = data_plasma_n4pe.loc[data_plasma_n4pe['ID_ADM'].duplicated(), 'ID_ADM']
dup_ID2 = data_plasma_n4pe.loc[data_plasma_n4pe['ID_ADM'].isin(dup_ID) & ~data_plasma_n4pe['timepoint'].isin(['end', 'base', 'mid']), 'ID_ADM']
data_plasma_n4pe.loc[data_plasma_n4pe['ID_ADM'].isin(dup_ID2) , ['Sample ID', 'ID_ADM', 'timepoint']].sort_values(['ID_ADM', 'timepoint'])

,Sample ID,ID_ADM,timepoint


## compare subjects listed in each sheet

In [26]:
# compare which subjects appear in each of the plasma sheets

print('missing from 231 but not 217:\n', data_plasma_pt217['ID_ADM'][~data_plasma_pt217['ID_ADM'].isin(data_plasma_pt231['ID_ADM'])])
print('missing from 231 but not n4pe:\n', data_plasma_n4pe['ID_ADM'][~data_plasma_n4pe['ID_ADM'].isin(data_plasma_pt231['ID_ADM'])])

print('missing from 217 but not 231:\n', data_plasma_pt231['ID_ADM'][~data_plasma_pt231['ID_ADM'].isin(data_plasma_pt217['ID_ADM'])])
print('missing from 217 but not n4pe:\n', data_plasma_n4pe['ID_ADM'][~data_plasma_n4pe['ID_ADM'].isin(data_plasma_pt217['ID_ADM'])])

print('missing from n4pe but not 217:\n', data_plasma_pt217['ID_ADM'][~data_plasma_pt217['ID_ADM'].isin(data_plasma_n4pe['ID_ADM'])])
print('missing from n4pe but not 231:\n', data_plasma_pt231['ID_ADM'][~data_plasma_pt231['ID_ADM'].isin(data_plasma_n4pe['ID_ADM'])])

# APE-767_14 has ptau217 but no ptau 231 or n4pe
# APE-767_13 has ptau 231 and n4pe but not ptau217
# APE-767_23 has ptau231 and n4pe but not ptau 217


missing from 231 but not 217:
 151    APE-767_14
Name: ID_ADM, dtype: object
missing from 231 but not n4pe:
 Series([], Name: ID_ADM, dtype: object)
missing from 217 but not 231:
 84    APE-767_13
92    APE-767_23
Name: ID_ADM, dtype: object
missing from 217 but not n4pe:
 155    APE-767_13
163    APE-767_23
Name: ID_ADM, dtype: object
missing from n4pe but not 217:
 151    APE-767_14
Name: ID_ADM, dtype: object
missing from n4pe but not 231:
 Series([], Name: ID_ADM, dtype: object)


## Merge all plasma data

In [27]:
data_plasma_all = pd.merge(
       data_plasma_pt217[['ID_ADM', 'timepoint', 'Subject_ID', 'pTau217 [ALZpath Ptau217 Quanterix] (pg/mL)', 'Subject_Label', 'Subject_Number']], 
       data_plasma_pt231[['ID_ADM', 'timepoint', 'Subject_ID', 'pTau231-3 [ptau181 inhouse] (pg/mL)', 'Subject_Label', 'Subject_Number']],
       on = ['ID_ADM', 'timepoint', 'Subject_Label', 'Subject_ID', 'Subject_Number'], how = 'outer')
data_plasma_all = pd.merge(
       data_plasma_all, 
       data_plasma_n4pe[['ID_ADM', 'timepoint', 'Subject_ID', 'Ab40 [Neuro 4-Plex E] (pg/mL)', 'Ab42 [Neuro 4-Plex E] (pg/mL)', 'GFAP [Neuro 4-Plex E] (pg/mL)', 'NFL [Neuro 4-Plex E] (pg/mL)', 
       'pTau181 [ptau181 inhouse] (pg/mL)', 'Subject_Label', 'Subject_Number']], 
       on = ['ID_ADM', 'timepoint', 'Subject_Label', 'Subject_ID', 'Subject_Number'], how = 'outer')

# rename plasma output to shorter name
data_plasma_all = data_plasma_all.rename(columns = {'pTau217 [ALZpath Ptau217 Quanterix] (pg/mL)':'pTau217',
 'pTau231-3 [ptau181 inhouse] (pg/mL)':'pTau231', 
       'Ab40 [Neuro 4-Plex E] (pg/mL)':'Ab40',
        'Ab42 [Neuro 4-Plex E] (pg/mL)':'Ab42', 
        'GFAP [Neuro 4-Plex E] (pg/mL)':'GFAP',
        'NFL [Neuro 4-Plex E] (pg/mL)':'NFL',
        'pTau181 [ptau181 inhouse] (pg/mL)':'pTau181'})

# replace invalid results with nan
data_plasma_all[['pTau217', 'pTau231', 'Ab40', 'Ab42', 'GFAP', 'NFL']] = data_plasma_all[['pTau217', 'pTau231', 'Ab40', 'Ab42', 'GFAP', 'NFL']].replace('<CalB |< CalB|>CalH', np.nan, regex = True).astype('float64')

# drop any rows missing all data
data_plasma_all = data_plasma_all.dropna(subset = ['pTau217', 'pTau231', 'Ab40', 'Ab42', 'GFAP', 'NFL', 'pTau181'], how = 'all')


# created Ab 42/40 ratio
data_plasma_all['Ab42_40'] = data_plasma_all['Ab42']/data_plasma_all['Ab40']

# created ptau 181/Ab42 ratio
data_plasma_all['pTau181_Ab42'] = data_plasma_all['pTau181']/data_plasma_all['Ab42']


In [28]:
# make log-transformed version of plasma data
plasma_columns = [ 'Ab40', 'Ab42', 'Ab42_40', 'GFAP','NFL',  'pTau181', 'pTau217',  'pTau231', 'pTau181_Ab42']

data_plasma_all[[s + '_log10' for s in plasma_columns]] = np.log10(data_plasma_all[plasma_columns])
log10_plasma_columns = [s + '_log10' for s in plasma_columns]

# Non-FDG imaging
Done : data_miscimaging_ril

## CV1 data: Not including in merge

In [29]:
data_CV1_ril = pd.read_excel(data_folder_loc + 'original\discrete\RIL CV1 scores baseline.xlsx')

In [30]:
# add CV1 data
data_CV1_ril = data_CV1_ril[['Subject ID', 'CV1']]
data_CV1_ril = data_CV1_ril.rename(columns = {'Subject ID':'Subject_ID'})
data_CV1_ril['timepoint'] = 'base'
data_CV1_ril['Subject_ID'] = data_CV1_ril['Subject_ID'].replace('RKF-APE','APE-792', regex = True).replace('RKF-RIL','RIL', regex = True)

## FreeSurfer Z-scores : not including in merge

In [31]:
# fix column names so consistent
data_freesurfer_ril_z.columns = data_freesurfer_ril_z.columns.str.replace(' |-', '_', regex = True)
data_freesurfer_ril_z

,Subject_ID,timepoint,Subject_Number,Subject_Label,ID_ADM,VOL_Precun_Lz,VOL_Precun_Rz,VOL_InfPar_Lz,VOL_InfPar_Rz,VOL_Hip_Lz,VOL_Hip_Rz,VOL_TotalGrayz,VOL_Inf_Mid_Fus_Temp_Lz,VOL_Inf_Mid_Fus_Temp_Rz,VOL__MidFront_Lz,VOL_MidFront_Rz
0,APE-792-018,base,18,APE-792,APE-792_18,2.4089,1.0677,2.8726,1.3681,-0.2931,-3.5883,0.7160,1.1947,-0.6769,-0.9422,-0.9125
1,APE-792-026,base,26,APE-792,APE-792_26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RIL-001,base,1,RIL,RIL_1,-2.8250,-3.7066,-3.8296,-3.7980,-3.6870,-4.5223,-4.6231,-3.5093,-3.0013,-1.5241,-1.5257
3,RIL-018,base,18,RIL,RIL_18,-1.1840,-1.3592,-0.5916,-0.9819,-1.6079,-1.4723,-2.0640,-0.9611,-0.9721,-0.4652,-0.8202
4,RIL-019,base,19,RIL,RIL_19,-1.4690,-0.9240,-1.5567,-2.8382,-1.9891,-4.5895,-2.6520,-2.6452,-4.6987,0.0129,-0.1003
5,RIL-025,base,25,RIL,RIL_25,-1.3238,-1.5855,-2.6004,-1.9377,-0.9533,-1.4046,-2.8153,-3.3171,-2.5013,-2.1960,-0.0345
6,RIL-027,base,27,RIL,RIL_27,-2.4374,-2.6135,-1.8129,-1.4716,-4.2807,-4.3285,-2.1903,-2.5943,-2.8022,-1.0669,-0.1598
7,RIL-028,base,28,RIL,RIL_28,-1.5652,-0.0258,-1.2263,-1.2089,-1.2888,-1.6167,-1.5828,-2.1921,-1.3475,-0.8987,-1.4046
8,APE-792-001,base,1,APE-792,APE-792_1,-0.9473,-0.8719,-0.3768,-1.2336,-2.0963,-1.9014,-1.3825,-2.4778,-0.9208,0.5267,-0.1854
9,APE-792-010,base,10,APE-792,APE-792_10,-2.8070,-3.2121,-3.7312,-5.1586,-2.9692,-3.6689,-4.6645,-4.0681,-5.6061,-0.4206,-0.5345


## new FreeSurfer Z-scores

In [32]:
data_fs_1 = pd.read_csv(data_folder_loc + "original/discrete/FSdiscreteDataMasks.csv", skiprows = 1)
data_fs_2 = pd.read_csv(data_folder_loc + "original/discrete/FSdiscreteDataMasks (2).csv", skiprows = 1)
data_fs_3 = pd.read_csv(data_folder_loc + "original/discrete/FSdiscreteDataMasks (3).csv", skiprows = 1)

data_fs = pd.merge(data_fs_1, data_fs_2, how = 'outer')
data_fs = pd.merge(data_fs, data_fs_3, how = 'outer')

data_fs = data_fs[['ImageNames', 'Subjects', 'Labels'] + data_fs.columns[data_fs.columns.str.contains('z')].to_list()]

data_fs = data_fs.rename(columns = {'Labels':'timepoint', 'Subjects':'Subject_ID'})

data_fs['timepoint'] = data_fs['timepoint'].replace({'m00':'base', 'm06':'end'})

data_fs['Subject_ID'] = data_fs['Subject_ID'].replace('RKF-APE','APE-792', regex = True).replace('RKF-RIL','RIL', regex = True)
data_fs['Subject_Label'] = data_fs['Subject_ID'].str[0:-4]
data_fs['Subject_Number'] = data_fs['Subject_ID'].str[-3:].astype('int')
data_fs['ID_ADM'] = data_fs['Subject_Label'] + '_' + data_fs['Subject_Number'].astype('str')

data_fs.columns = data_fs.columns.str.replace('^RV', 'VOL', regex = True)

In [33]:
data_fs.columns

Index(['ImageNames', 'Subject_ID', 'timepoint', 'VOL_Ventricles_Lz',
       'VOL_Ventricles_Rz', 'VOL_Putamen_Lz', 'VOL_Putamen_Rz',
       'VOL_ParaHip_Lz', 'VOL_ParaHip_Rz', 'VOL_Fusi_Lz', 'VOL_Fusi_Rz',
       'VOL_InfTemp_Lz', 'VOL_InfTemp_Rz', 'VOL_MidTemp_Lz', 'VOL_MidTemp_Rz',
       'VOL_SupTemp_Lz', 'VOL_SupTemp_Rz', 'VOL_Precun_Lz', 'VOL_Precun_Rz',
       'VOL_InfPar_Lz', 'VOL_InfPar_Rz', 'VOL_ParaPostCentr_Lz',
       'VOL_ParaPostCentr_Rz', 'VOL_SupraMarg_Lz', 'VOL_SupraMarg_Rz',
       'VOL_SupPar_Lz', 'VOL_SupPar_Rz', 'VOL_OrbitFront_Lz',
       'VOL_OrbitFront_Rz', 'VOL_Insula_Lz', 'VOL_Insula_Rz',
       'VOL_InfFront_Lz', 'VOL_InfFront_Rz', 'VOL_MidFront_Lz',
       'VOL_MidFront_Rz', 'VOL_SupFront_Lz', 'VOL_SupFront_Rz',
       'VOL_PrecFront_Lz', 'VOL_PrecFront_Rz', 'VOL_LatOcc_Lz',
       'VOL_LatOcc_Rz', 'VOL_Lingual_Lz', 'VOL_Lingual_Rz', 'VOL_Cuneus_Lz',
       'VOL_Cuneus_Rz', 'VOL_Pericalc_Lz', 'VOL_Pericalc_Rz',
       'VOL_AntCingulate_Lz', 'VOL_AntCingulate

In [34]:
data_fs_base = data_fs[data_fs['timepoint'].isin(['base'])]

In [35]:
data_fs_base.sort_values('Subject_ID')

,ImageNames,Subject_ID,timepoint,VOL_Ventricles_Lz,VOL_Ventricles_Rz,VOL_Putamen_Lz,VOL_Putamen_Rz,VOL_ParaHip_Lz,VOL_ParaHip_Rz,VOL_Fusi_Lz,...,VOL_Precun_InfPar_Rz,VOL_Precun_InfPar_Supramarg_Lz,VOL_Precun_InfPar_Supramarg_Rz,VOL_LatOccLingCun_Lz,VOL_LatOccLingCun_Rz,VOL_InfParSupra_Lz,VOL_InfParSupra_Rz,Subject_Label,Subject_Number,ID_ADM
30,RKF-APE-001_m00_na_NuNtv.nii,APE-792-001,base,-0.3131,-0.4806,0.0027,0.0027,-1.8821,-0.7909,0.8132,...,-0.7533,0.0446,-1.0777,-0.8096,-0.0913,0.4494,-0.9526,APE-792,1,APE-792_1
31,RKF-APE-005_m00_na_NuNtv.nii,APE-792-005,base,1.1719,1.5888,0.0042,0.0040,-1.0593,0.5906,-1.5598,...,1.7446,0.2246,0.8995,-0.1103,-0.0475,0.4340,1.3592,APE-792,5,APE-792_5
32,RKF-APE-007_m00_na_NuNtv.nii,APE-792-007,base,2.4318,2.5542,0.0027,0.0028,0.2580,-0.2106,-1.6225,...,-0.8224,-3.3556,-1.1536,-1.2565,-0.5410,-2.9313,-1.0190,APE-792,7,APE-792_7
33,RKF-APE-009_m00_na_NuNtv.nii,APE-792-009,base,0.1241,0.3749,0.0034,0.0031,-0.7399,0.8021,-1.3450,...,-1.6374,-0.6476,-1.2882,0.0235,0.4753,-0.3107,-1.1620,APE-792,9,APE-792_9
34,RKF-APE-010_m00_na_NuNtv.nii,APE-792-010,base,1.6262,2.2786,0.0036,0.0033,0.2835,-2.4636,-1.9420,...,-5.1004,-4.1953,-5.8106,-3.0170,-2.9015,-4.2058,-5.9207,APE-792,10,APE-792_10
35,RKF-APE-011_m00_na_NuNtv.nii,APE-792-011,base,2.3133,2.5338,0.0034,0.0037,0.6001,0.5734,0.3665,...,1.0680,0.4938,0.3798,0.4534,0.2726,0.5152,0.4555,APE-792,11,APE-792_11
36,RKF-APE-013_m00_na_NuNtv.nii,APE-792-013,base,0.8269,0.0423,0.0031,0.0032,0.3109,1.1167,-0.9576,...,0.3684,-0.3865,-0.0566,-0.9681,0.3452,-0.4780,-0.2009,APE-792,13,APE-792_13
37,RKF-APE-016_m00_na_NuNtv.nii,APE-792-016,base,1.6699,1.9596,0.0030,0.0029,-1.1385,-2.1839,-1.1285,...,-1.7137,-1.4418,-1.4397,1.5605,1.0414,-1.2464,-1.4865,APE-792,16,APE-792_16
38,RKF-APE-018_m00_na_NuNtv.nii,APE-792-018,base,3.3489,3.2666,0.0026,0.0025,-1.1387,-1.6459,1.6909,...,0.8916,2.4065,1.3026,-1.3840,-1.3331,1.9902,1.3179,APE-792,18,APE-792_18
39,RKF-APE-020_m00_na_NuNtv.nii,APE-792-020,base,1.4591,0.9323,0.0040,0.0041,-1.3612,-0.9960,0.5117,...,1.3106,1.1018,1.6324,2.1892,1.6024,1.1363,1.7546,APE-792,20,APE-792_20


In [36]:
temp = pd.merge(data_fs_base, data_freesurfer_ril_z, how = 'outer', on = ['ID_ADM', 'timepoint', 'Subject_ID', 'Subject_Label', 'Subject_Number'], suffixes=('', '.old'))

In [37]:
temp.columns[temp.columns.str.contains('.old')]

Index(['VOL_Precun_Lz.old', 'VOL_Precun_Rz.old', 'VOL_InfPar_Lz.old',
       'VOL_InfPar_Rz.old', 'VOL_Hip_Lz.old', 'VOL_Hip_Rz.old',
       'VOL_TotalGrayz.old', 'VOL_Inf_Mid_Fus_Temp_Lz.old',
       'VOL_Inf_Mid_Fus_Temp_Rz.old', 'VOL_MidFront_Rz.old'],
      dtype='object')

In [38]:
a = 'VOL_Precun_Lz'
temp.loc[temp[a ] != temp[a + '.old'], [a , a + '.old'] ].dropna()

,VOL_Precun_Lz,VOL_Precun_Lz.old
0,-0.805400,-1.3284
1,-0.520900,-1.4563
3,-3.180400,-3.4796
4,-1.079400,-1.1845
6,-1.351800,-1.1840
7,-1.794800,-1.4690
8,0.029600,0.2350
9,-0.953100,-1.4021
11,-1.030000,-1.3238
12,-2.838900,-2.4374


## FreeSurfer from Randy DB: not including in merge

In [39]:
data_freesurfer_ril_randy = data_freesurfer_ril_randy[data_freesurfer_ril_randy['Aseg_vol_B_WM_HYPOINT'].notna() & ~data_freesurfer_ril_randy['Aseg_vol_B_WM_HYPOINT'].isin(['pending'])]
data_freesurfer_ril_randy = data_freesurfer_ril_randy.drop(columns= ['INDEX-1.1', 'INDEX-2.1'])
data_freesurfer_ril_randy

,timepoint,ID_ADM,Subject_Label,Subject_Number,MRI Visit Date,7-Character machine UID,MRI ReName,SPM12 Source Mat file,GM,WM,...,Aseg_vol_R_CHORPLEX,Aseg_vol_R_HIP,Aseg_vol_R_INF_LATVENT,Aseg_vol_R_LATVENT,Aseg_vol_R_PALLIDUM,Aseg_vol_R_PUTAMEN,Aseg_vol_R_THALAMUS,Aseg_vol_R_VENTRALDC,Aseg_vol_R_VESSEL,Aseg_vol_B_WM_HYPOINT
1,base,APE-792_1,APE-792,1,2013-11-25 00:00:00,e010542,RKF-APE-001__m00__T1-a__20131125__e010542.nii,'M:\RKF_spm12_061119\seg8-mat\RKF-APE-001__m00...,588.8171,408.9409,...,540.6,3432.0,177.4,13629.8,1787.3,3335.8,5590.7,3232.2,56.1,2522.7
2,mid,APE-792_1,APE-792,1,2014-04-03 00:00:00,e010678,RKF-APE-001__m04__T1-b__20140403__e010678.nii,'M:\RKF_spm12_061119\seg8-mat\RKF-APE-001__m04...,599.6905,401.7833,...,613.1,3341.3,216.2,14527.6,1927.6,3496.8,5631.8,3077.9,85.1,1994.7
3,end,APE-792_1,APE-792,1,2014-07-01 00:00:00,n010799,RKF-APE-001__m06__T1-c__20140701__n010799.nii,M:\RKF_spm12_061119\seg8-mat\RKF-APE-001__m06_...,585.4306,417.9093,...,656.8,3252.9,277.7,13762.8,1863.5,3155.3,5452.4,3006.3,71.7,2563.2
7,base,APE-792_5,APE-792,5,2014-04-28 00:00:00,e010707,RKF-APE-005__m00__T1-a__20140428__e010707.nii,'M:\RKF_spm12_061119\seg8-mat\RKF-APE-005__m00...,544.4066,461.9599,...,1043.2,3796.4,1580.5,32538.3,2436.2,5944.4,6849.1,3295.2,113.1,9032.4
9,end,APE-792_5,APE-792,5,2014-11-10 00:00:00,n111826,RKF-APE-005__m06__T1-c__20141110__n111826.nii,M:\RKF_spm12_061119\seg8-mat\RKF-APE-005__m06_...,535.6529,460.1551,...,938.9,4137.5,1774.0,39128.0,1924.6,4726.1,6266.6,3194.0,11.3,10789.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,base,RIL_22,RIL,22,2019-05-28 00:00:00,e013678,RKF-RIL-022__m00__T1-a__20190528__e013678.nii,'M:\RKF_spm12_061119\seg8-mat\RKF-RIL-022__m00...,537.7832,369.4351,...,637.8,3029.2,683.6,17338.3,1183.1,3626.9,5403.8,3161.9,8.6,3598.6
289,base,RIL_24,RIL,24,2019-07-24 00:00:00,e013826,RKF-RIL-024__m00__T1-a__20190724__e013826.nii,'J:\RIL_S_2024A_spm12_072419\RKF-RIL-024__m00_...,509.7671,360.4517,...,593.5,2926.7,866.8,17898.2,1147.6,3564.5,5231.5,2992.4,27.0,2454.2
293,base,RIL_25,RIL,25,2019-06-06 00:00:00,e013707,RKF-RIL-025__m00__T1-a__20190607__e013707.nii,'M:\RKF_spm12_061119\seg8-mat\RKF-RIL-025__m00...,574.0856,395.8961,...,432.7,3266.4,747.8,16422.1,1867.4,4285.1,6049.4,3353.1,83.1,2944.9
305,base,RIL_28,RIL,28,2019-07-26 00:00:00,UNK,RKF-RIL-028__m00__T1-a__20190726__e013839.nii,M:\RKF-RIL-028A_spm12_072719\o_RKF-RIL-028__m0...,606.6235,388.7486,...,722.9,2963.7,1327.5,24639.3,1361.4,3691.9,5454.8,3210.1,53.7,3795.7


## Non-Riluzole MRS data : not including in merge

In [40]:
data_MRS_nonril = pd.read_excel(data_folder_loc + "original\discrete\FINAL_SDB data.xlsx", sheet_name = 'Sheet1', skiprows = 1) # control biomarkers

c:\Users\Lauren.Koenig\.conda\envs\Lauren_export_conda_20250102\lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed


In [41]:
# drop the empty rows (read in because of extra rows that defined the acronyms)
data_MRS_nonril = data_MRS_nonril[data_MRS_nonril['ID'].notna()]

# separate out the Subject_ID's prefix and #
data_MRS_nonril['Subject_Label'] =   data_MRS_nonril['STUDY'] #'CCH'
data_MRS_nonril['Subject_Number'] = data_MRS_nonril['ID'].astype('int')

# recombine prefix and label into a consistent ID
data_MRS_nonril['ID_ADM'] = data_MRS_nonril['Subject_Label'] + '_' + data_MRS_nonril['Subject_Number'].astype('str')

# look at # of subjects
print('\nSDB', data_MRS_nonril['Subject_Label'].isna().value_counts())
print('\nSDB', data_MRS_nonril['Subject_Label'].value_counts())
print('\nSDB', data_MRS_nonril.drop_duplicates(subset = 'ID_ADM')['Subject_Label'].value_counts())



SDB False    35
True      7
Name: Subject_Label, dtype: int64

SDB DNP     31
PDSS     2
SNPR     1
NP       1
Name: Subject_Label, dtype: int64

SDB DNP     31
PDSS     2
SNPR     1
NP       1
Name: Subject_Label, dtype: int64


## combine non-FDG imaging data

In [42]:
data_freesurfer_ril_z.columns[data_freesurfer_ril_z.columns.isin(data_freesurfer_ril_randy.columns)]

Index(['timepoint', 'Subject_Number', 'Subject_Label', 'ID_ADM'], dtype='object')

In [43]:
data_fs.columns[data_fs.columns.isin(data_freesurfer_ril_randy.columns)]

Index(['timepoint', 'Subject_Label', 'Subject_Number', 'ID_ADM'], dtype='object')

In [44]:
data_miscimaging_ril = pd.merge(data_fs.drop(columns = ['Subject_ID']), data_freesurfer_ril_randy, how = 'outer')

data_miscimaging_ril

,ImageNames,timepoint,VOL_Ventricles_Lz,VOL_Ventricles_Rz,VOL_Putamen_Lz,VOL_Putamen_Rz,VOL_ParaHip_Lz,VOL_ParaHip_Rz,VOL_Fusi_Lz,VOL_Fusi_Rz,...,Aseg_vol_R_CHORPLEX,Aseg_vol_R_HIP,Aseg_vol_R_INF_LATVENT,Aseg_vol_R_LATVENT,Aseg_vol_R_PALLIDUM,Aseg_vol_R_PUTAMEN,Aseg_vol_R_THALAMUS,Aseg_vol_R_VENTRALDC,Aseg_vol_R_VESSEL,Aseg_vol_B_WM_HYPOINT
0,RKF-RIL-011_m00_na_NuNtv.nii,base,0.5389,1.6890,0.0033,0.0032,-0.5001,-1.3911,0.0620,-0.7349,...,924.2,2554.3,1618.8,33283.0,1181.7,3679.8,4677.7,2953.4,83.3,2999.6
1,RKF-RIL-012_m00_na_NuNtv.nii,base,1.2721,2.0021,0.0038,0.0037,-1.7391,-2.4908,-3.4362,-4.6081,...,1054.7,2313.8,3041.0,29821.4,948.1,3735.2,4958.1,3129.1,28.6,4353.3
2,RKF-RIL-013_m00_na_NuNtv.nii,base,0.8790,1.3316,0.0029,0.0027,-1.6888,-3.2479,-1.3316,-2.8712,...,701.9,2641.5,1604.8,23218.9,1101.9,3376.4,5527.6,3521.7,45.2,3461.5
3,RKF-RIL-014_m00_na_NuNtv.nii,base,0.8540,0.6604,0.0029,0.0030,-1.8576,-1.0499,-1.5129,-0.9954,...,608.4,3400.3,1004.8,21868.4,1222.1,3998.7,5324.4,3629.1,30.2,3682.4
4,RKF-RIL-015_m00_na_NuNtv.nii,base,0.8531,1.0264,0.0037,0.0037,-1.4632,-0.8182,-2.2422,-0.8366,...,865.1,2367.6,2104.1,24462.4,1611.8,4194.9,5624.3,3053.5,139.6,23159.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,NaN,mid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,676.4,2604.5,1588.7,23816.8,1139.3,3380.7,5463.3,3325.3,32.9,3983.5
78,NaN,mid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,588.4,3000.5,1011.7,22268.8,1262.1,3857.6,5142.5,3439.4,68.3,3883.4
79,NaN,mid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1151.3,2296.0,3287.5,31101.1,1118.4,3488.7,5000.4,2941.5,53.2,5467.6
80,NaN,mid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,676.4,1922.6,2172.9,23165.5,899.0,2769.5,4240.4,3020.4,55.2,6766.4


# Clinical Data
Done : data_clin

## Riluzole ppts

In [45]:
data_clin_ril_orig = pd.read_excel(data_folder_loc + "original\discrete\ALL riluzole_data.xlsx")

c:\Users\Lauren.Koenig\.conda\envs\Lauren_export_conda_20250102\lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed


In [46]:
# keep just the useful columns
#data_clin_ril_orig = data_clin_ril_orig[['Subject_ID', 'visit',
 #'Group',
 #'Site',
 #'Race_Ethnicity',
 #'Education_years',
 #'age',
 #'gender',
 #'APOE', 'Screening_MMSE', 'adascogtotal', 'bvrt', 'dstotal','tma','tmb',  'cowattotal', 'cdrtotal', 'cdrsum', 'adltotal', 'npitotal', 'gds']].drop_duplicates()

data_clin_ril_orig = data_clin_ril_orig.rename(columns = {'visit':'timepoint'})
data_clin_ril_orig['timepoint'] = data_clin_ril_orig['timepoint'].replace({0:'base', 6:'end', 3:'mid'})

# drop extra 0 in ID to make consistent with other sheet's IDs
data_clin_ril_orig['Subject_ID'] = data_clin_ril_orig['Subject_ID'].replace('APE-0792-','APE-792-',   regex = True)


data_clin_ril_orig = data_clin_ril_orig.rename(columns = {'Screening_MMSE':'MMSE', 'gender':'sex', 'Race_Ethnicity':'race'})
data_clin_ril_orig['apoe4_carrier'] = data_clin_ril_orig['apoe4_carrier'].replace('Unk', np.nan)

data_clin_ril_orig['Diagnosis'] = 'AD'
data_clin_ril_orig['Subject_Label'] = data_clin_ril_orig['Subject_ID'].str[0:-4]
data_clin_ril_orig['Subject_Number'] = data_clin_ril_orig['Subject_ID'].str[-3:].astype('int')
data_clin_ril_orig['ID_ADM'] = data_clin_ril_orig['Subject_Label']  + '_' + data_clin_ril_orig['Subject_Number'].astype('str') 
data_clin_ril_orig['sex'] = data_clin_ril_orig['sex'].replace({'Female':'F', 'Male':'M'})
data_clin_ril_orig['race'] = data_clin_ril_orig['race'].replace({'Black/AA':'Black', 'Black/NH':'Black' })


## Riluzole Ppts from Randy's Database

In [47]:
data_clin_randy = data_clin_randy.replace('UNK', np.nan).replace('O', np.nan)
data_clin_randy['age'] = (data_clin_randy['age'].astype('str') + ' ').str.split(' ', expand = True)[0].astype('float64')


data_clin_randy['Diagnosis'] = 'AD'

data_clin_randy

,timepoint,ID_ADM,Subject_Label,Subject_Number,age,sex,Diagnosis
1,base,APE-792_1,APE-792,1,76.0,M,AD
2,mid,APE-792_1,APE-792,1,76.0,M,AD
3,end,APE-792_1,APE-792,1,76.0,M,AD
7,base,APE-792_5,APE-792,5,81.0,M,AD
8,mid,APE-792_5,APE-792,5,81.0,M,AD
...,...,...,...,...,...,...,...
312,mid,RIL_29,RIL,29,74.0,F,AD
313,end,RIL_29,RIL,29,74.0,F,AD
317,base,RIL_31,RIL,31,72.0,M,AD
318,mid,RIL_31,RIL,31,72.0,M,AD


## demographics given to us piecemeal by Ana

'APE-792_22': 67 yo F
'APE-792_39': was excluded by neuropsychological testing as not meeting criteria for AD
'APE-792_60': 84yo F
'RIL_5': 64 yo F
'APE-792_23' was excluded because he had a more Lewy Body Disease pattern. He is male and age 70yo
'APE-792_35' was excluded because Dawn said the FDG Pet was not characteristic of AD. Female and 75yo
'RIL_21' was excluded as not a characteristic pattern of AD on FDG PET. Female and 72 yo.

In [48]:
['APE-792_39', 'APE-792_23', 'APE-792_35', 'RIL_21']

['APE-792_39', 'APE-792_23', 'APE-792_35', 'RIL_21']

In [49]:
data_clin_ana = pd.DataFrame({
    'ID_ADM': ['APE-792_22', 'APE-792_39', 'APE-792_60', 'RIL_5', 'APE-792_23', 'APE-792_35', 'RIL_21'],
    'age': [67, np.nan, 84, 64, 70, 72, 72],
    'sex': ['F', np.nan, 'F', 'F', 'M', 'F', 'F'],
    'Diagnosis': ['AD', 'not AD', 'AD', 'AD', 'LBD pattern', 'not AD', 'not AD'],

})

data_clin_ana['Subject_Label'] = data_clin_ana['ID_ADM'].str.split('_', expand = True)[0]
data_clin_ana['Subject_Number'] = data_clin_ana['ID_ADM'].str.split('_', expand = True)[1].astype('int')

## combine Riluzole participants

In [50]:
# just keep those that aren't already in the previous clinical data
data_clin_randy_subset = data_clin_randy[~data_clin_randy['ID_ADM'].isin(data_clin_ril_orig['ID_ADM'])].dropna(subset = ['age', 'sex'])


In [51]:
data_clin_ril = pd.merge(data_clin_ril_orig, data_clin_randy_subset, how = 'outer')
data_clin_ril

,Subject_ID,Group,Site,Study_Completed,race,Education_years,MMSE,age,sex,APOE,...,RH_Internal_Water,RH_GLX_W,RH_GABA_W,RH_CHO_W,RH_CR_W,RH_NAA_W,Diagnosis,Subject_Label,Subject_Number,ID_ADM
0,APE-792-001,Placebo,Rockefeller,Y,White/NH,14.0,26.0,76.0,M,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,AD,APE-792,1,APE-792_1
1,APE-792-001,Placebo,Rockefeller,Y,White/NH,14.0,26.0,76.0,M,23.0,...,2.206000e+12,0.001087,0.001559,0.010789,0.010839,0.014166,AD,APE-792,1,APE-792_1
2,APE-792-001,Placebo,Rockefeller,Y,White/NH,14.0,26.0,76.0,M,23.0,...,2.319000e+12,0.001022,0.001136,0.012130,0.011298,0.017382,AD,APE-792,1,APE-792_1
3,APE-792-005,Riluzole,Rockefeller,Y,White/NH,18.0,23.0,81.0,M,33.0,...,1.786000e+12,0.001701,0.002053,0.011976,0.010235,0.016725,AD,APE-792,5,APE-792_5
4,APE-792-005,Riluzole,Rockefeller,Y,White/NH,18.0,23.0,81.0,M,33.0,...,1.917000e+12,0.001584,0.002073,0.011205,0.009077,0.016740,AD,APE-792,5,APE-792_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69.0,M,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,AD,APE-792,71,APE-792_71
130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,M,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,AD,RIL,13,RIL_13
131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.0,M,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,AD,RIL,13,RIL_13
132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,F,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,AD,RIL,17,RIL_17


In [52]:
data_clin_ril = pd.merge(data_clin_ril, data_clin_ana[~data_clin_ana['ID_ADM'].isin(data_clin_ril['ID_ADM'])], how = 'outer')
data_clin_ril

,Subject_ID,Group,Site,Study_Completed,race,Education_years,MMSE,age,sex,APOE,...,RH_Internal_Water,RH_GLX_W,RH_GABA_W,RH_CHO_W,RH_CR_W,RH_NAA_W,Diagnosis,Subject_Label,Subject_Number,ID_ADM
0,APE-792-001,Placebo,Rockefeller,Y,White/NH,14.0,26.0,76.0,M,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,AD,APE-792,1,APE-792_1
1,APE-792-001,Placebo,Rockefeller,Y,White/NH,14.0,26.0,76.0,M,23.0,...,2.206000e+12,0.001087,0.001559,0.010789,0.010839,0.014166,AD,APE-792,1,APE-792_1
2,APE-792-001,Placebo,Rockefeller,Y,White/NH,14.0,26.0,76.0,M,23.0,...,2.319000e+12,0.001022,0.001136,0.012130,0.011298,0.017382,AD,APE-792,1,APE-792_1
3,APE-792-005,Riluzole,Rockefeller,Y,White/NH,18.0,23.0,81.0,M,33.0,...,1.786000e+12,0.001701,0.002053,0.011976,0.010235,0.016725,AD,APE-792,5,APE-792_5
4,APE-792-005,Riluzole,Rockefeller,Y,White/NH,18.0,23.0,81.0,M,33.0,...,1.917000e+12,0.001584,0.002073,0.011205,0.009077,0.016740,AD,APE-792,5,APE-792_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.0,F,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,AD,APE-792,60,APE-792_60
137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,F,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,AD,RIL,5,RIL_5
138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,M,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,LBD pattern,APE-792,23,APE-792_23
139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,F,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,not AD,APE-792,35,APE-792_35


## add education
APE-0792-022 Information not found  

In [53]:
data_clin_ril.loc[data_clin_ril['ID_ADM'].isin(['RIL_13']), 'Education_years'] = 19
data_clin_ril.loc[data_clin_ril['ID_ADM'].isin(['RIL_17']), 'Education_years'] = 19
data_clin_ril.loc[data_clin_ril['ID_ADM'].isin(['RIL_24']), 'Education_years'] = 11
data_clin_ril.loc[data_clin_ril['ID_ADM'].isin(['APE-792_9']), 'Education_years'] = 16 # estimated from 'Completed College Degree - BA'
data_clin_ril.loc[data_clin_ril['ID_ADM'].isin(['APE-792_71']), 'Education_years'] = 11 # estimated from 'Did not complete High School'
data_clin_ril.loc[data_clin_ril['ID_ADM'].isin(['APE-792_65']), 'Education_years'] = 18 # estimated from 'Completed MBA'

## ADRC ppts

In [54]:
data_clin_adrc_orig = pd.read_excel(data_folder_loc + "original\discrete\Clinical_Data_AD_Biomarkers_20240318.xlsx")

In [55]:
# keep just the useful columns
#data_clin_adrc_orig = data_clin_adrc_orig[['Subject_ID',
# 'Visit_#',
# 'Alt_ID',
# 'Draw_Date',
# 'Age_at_Draw',
# 'Sex_reported',
# 'Race_reported',
# 'Diagnosis',
# 'Diagnosis_if_other',
# 'APOE']]


# change Subject_ID to the alternative ID in order to match the IDs used in plasma datasheets
data_clin_adrc_orig.loc[data_clin_adrc_orig['Alt_ID'].isin(['ADRC11068-01', 'ADRC52464-02', 'ADRC48043', 'ADRC55745', 'ADRC2800', 'ADRC93371', 'ADRC63118', 'BC0043']), 'Subject_ID'] = data_clin_adrc_orig.loc[data_clin_adrc_orig['Alt_ID'].isin([
    'ADRC11068-01', 'ADRC52464-02', 'ADRC48043', 'ADRC55745', 'ADRC2800', 'ADRC93371', 'ADRC63118', 'BC0043']), 'Alt_ID']

# separate out the Subject_ID's prefix and #
data_clin_adrc_orig['Subject_Label'] = data_clin_adrc_orig['Subject_ID'].str.replace('\d+', '', regex = True).str.replace('-', '', regex = True)
data_clin_adrc_orig['Subject_Number'] = data_clin_adrc_orig['Subject_ID'].str.replace('[A-Z]+', '', regex = True).str.split('-', expand= True)[0].astype('int')   

# drop the couple RIL subjects that are in here for some reason
data_clin_adrc_orig = data_clin_adrc_orig[~data_clin_adrc_orig['Subject_Label'].isin(['RIL'])]

# recombine prefix and label into a consistent ID
data_clin_adrc_orig['ID_ADM'] = data_clin_adrc_orig['Subject_Label'] + '_' + data_clin_adrc_orig['Subject_Number'].astype('str')

# look at # of subjects
print('\nAD_bio', data_clin_adrc_orig['Subject_Label'].isna().value_counts())
print('\nAD_bio', data_clin_adrc_orig['Subject_Label'].value_counts())
print('\nAD_bio', data_clin_adrc_orig.drop_duplicates(subset = 'ID_ADM')['Subject_Label'].value_counts())

data_clin_adrc_orig['Subject_ID'] = data_clin_adrc_orig['Subject_ID'].replace('RIL', 'RIL-', regex = True)

data_clin_adrc_orig['Subj_Visit'] = data_clin_adrc_orig['Subject_ID'] + '_' + data_clin_adrc_orig['Visit_#'].astype('str')

data_clin_adrc_orig['Race_reported'] = data_clin_adrc_orig['Race_reported'].replace({'White':'White/NH', 'African_American':'Black', 'Hispanic':'Hispanic/Latino'})
data_clin_adrc_orig['Sex_reported'] = data_clin_adrc_orig['Sex_reported'].replace({'Female':'F', 'Male':'M'})



AD_bio False    65
Name: Subject_Label, dtype: int64

AD_bio CCH      35
AANDS    22
ADRC      7
BC        1
Name: Subject_Label, dtype: int64

AD_bio CCH      35
AANDS    22
ADRC      7
BC        1
Name: Subject_Label, dtype: int64


## ADRC ppts - additional data sent later

In [56]:
data_clin_adrc_add = pd.read_excel(data_folder_loc + "original\discrete\AANDS_Metadata_Dr.Pereira_20240423.xlsx")

In [57]:
#rename columns so consistent
data_clin_adrc_add = data_clin_adrc_add.rename(columns = {'Age_at_draw':'Age_at_Draw', 'Acenstry_GSA':'Ancestry_gsa'})

# add combined visit, subject ID
data_clin_adrc_add['Subj_Visit'] = data_clin_adrc_add['Subject_ID'] + '_' + data_clin_adrc_add['Visit_#'].astype('str')
data_clin_adrc_add['Race_reported'] = data_clin_adrc_add['Race_reported'].replace({'White':'White/NH', 'African_American':'Black', 'Hispanic/Latino & Other (Mixed)':'Hispanic/Latino', 'African American & Hispanic/Latino':'Other'})
data_clin_adrc_add['Sex_reported'] = data_clin_adrc_add['Sex_reported'].replace({'Female':'F', 'Male':'M'})


## merge in new data without losing any original data for ADRC participants

In [58]:
# separate the subjects that have new data from those that don't
subjects_with_add_data = data_clin_adrc_orig[data_clin_adrc_orig['Subj_Visit'].isin(data_clin_adrc_add['Subj_Visit'])]
data_clin_adrc_orig = data_clin_adrc_orig[~data_clin_adrc_orig['Subj_Visit'].isin(data_clin_adrc_add['Subj_Visit'])]

# separate out which columns the new data has and those it doesn't - need to save the columns that it doesn't have separately 
cols_to_replace = data_clin_adrc_orig.columns[data_clin_adrc_orig.columns.isin(data_clin_adrc_add.columns)]
cols_to_save = data_clin_adrc_orig.columns[~data_clin_adrc_orig.columns.isin(data_clin_adrc_add.columns)].to_list() + ['Subj_Visit']

# just the subjects IDs and the columns not present in the new data - will replace all other columns with new version
subjects_with_add_data_save = subjects_with_add_data[cols_to_save]

# merge new data with the columns only present in old data
data_clin_adrc_add = pd.merge(data_clin_adrc_add, subjects_with_add_data_save, how = 'outer', on = ['Subj_Visit'], suffixes=('.new', '.ADbio'))

# reorder columns to match
data_clin_adrc_add = data_clin_adrc_add[data_clin_adrc_orig.columns]

#combine old and new data
data_clin_adrc = pd.concat([data_clin_adrc_orig, data_clin_adrc_add])



## Sleep ppts (just using age and sex)

In [59]:
data_clin_sleep = pd.read_excel(data_folder_loc + 'original\discrete\FINAL_SDB data analysis_all_with age.xlsx', skiprows = 1)

c:\Users\Lauren.Koenig\.conda\envs\Lauren_export_conda_20250102\lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed


In [60]:
data_clin_sleep = data_clin_sleep[['ID', 'age ', 'sex']].dropna(how = 'all')

data_clin_sleep = data_clin_sleep.rename(columns = {'age ':'Age_at_Draw', 'sex':'Sex_reported', 'ID':'Subject_Number'})
data_clin_sleep['Subject_Label'] = 'APE-767'


data_clin_sleep['Subject_Number'] = data_clin_sleep['Subject_Number'].astype('int')  

data_clin_sleep['ID_ADM'] = data_clin_sleep['Subject_Label'] + '_' + data_clin_sleep['Subject_Number'].astype('str')

data_clin_sleep['ID_ADM'] = data_clin_sleep['Subject_Label'] + '_' + data_clin_sleep['Subject_Number'].astype('str')

data_clin_sleep['Sex_reported'] = data_clin_sleep['Sex_reported'].replace({'Female':'F', 'Male':'M'})
data_clin_sleep['Subject_ID'] = data_clin_sleep['Subject_Label'] + "-" + data_clin_sleep['Subject_Number'].astype('int').apply(lambda x: str(x).zfill(3))

## Combine all non-RIL clinical data

In [61]:
data_clin_nonril = pd.merge(data_clin_adrc, data_clin_sleep, how = 'outer')

In [62]:
data_clin_nonril['Group'] = 'Control'
data_clin_nonril['apoe4_carrier'] = data_clin_nonril['APOE'].astype('str').replace({'23.0':'No', '24.0':'Yes', '33.0':'No', '34.0':'Yes', '44.0':'Yes', 'nan':np.nan})

data_clin_nonril = data_clin_nonril.rename(columns = {'CDR_last':'cdrtotal', 'Age_at_Draw':'age', 
'Sex_reported':'sex', 'Race_reported':'race'})

## combine all clinical data

In [63]:
# don't need any columns that don't exist for the Riluzole subjects
data_clin_nonril = data_clin_nonril.drop(columns=['Visit_#', 'Draw_Date', 'Ancestry_reported', 'Ancestry_gsa',
       'Notes_Diagnosis', 'Family_History_yesno',
       'Fam_History_Specifics', 'GBA_yesno', 'GBA_mut', 'LRRK2_yesno',
       'LRRK2_mut', 'Comorbidities', 'Age_of_onset', 'Amyloid_yesno', 'MOCA',
       'MOCA_date', 'MMSE_date', 'Diabetes_yesno', 'Stroke_yesno',
       'Hypertension_yesno', 'Depression_yesno', 'Drinker_yesno',
       'Current_Smoker_yesno'])
       
data_clin = pd.merge(data_clin_ril, data_clin_nonril, how = 'outer')

In [64]:
# separate into age decades
data_clin['age_decade'] = pd.cut(data_clin['age'], [40, 50, 60, 70, 80, 90, 100]).astype('str').replace('nan', np.nan).astype('category')

# separate into age decades2
data_clin['age_decade2'] = pd.cut(data_clin['age'], [50, 70, 80, 100]).astype('str').replace('nan', np.nan).astype('category')

# only keep alt id where it's actually a different ID than the subject ID
data_clin.loc[data_clin['Alt_ID'] == data_clin['Subject_ID'], 'Alt_ID'] = np.nan

data_clin.loc[data_clin['timepoint'].isna(), 'timepoint'] = 'not part of trial'

# combine all non-FDG data

In [65]:
data_no_fdg = pd.merge(data_clin.drop(columns = ['Subject_ID']), data_plasma_all.drop(columns = ['Subject_ID']), how = 'outer', on = ['timepoint', 'ID_ADM', 'Subject_Label', 'Subject_Number'])
data_no_fdg

,Group,Site,Study_Completed,race,Education_years,MMSE,age,sex,APOE,apoe4_status,...,pTau181_Ab42,Ab40_log10,Ab42_log10,Ab42_40_log10,GFAP_log10,NFL_log10,pTau181_log10,pTau217_log10,pTau231_log10,pTau181_Ab42_log10
0,Placebo,Rockefeller,Y,White/NH,14.0,26.0,76.0,M,23.0,Non-carrier,...,NaN,NaN,NaN,NaN,2.041393,1.525045,1.252853,-0.244125,1.788845,NaN
1,Placebo,Rockefeller,Y,White/NH,14.0,26.0,76.0,M,23.0,Non-carrier,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Placebo,Rockefeller,Y,White/NH,14.0,26.0,76.0,M,23.0,Non-carrier,...,NaN,NaN,NaN,NaN,NaN,NaN,0.551450,NaN,1.575309,NaN
3,Riluzole,Rockefeller,Y,White/NH,18.0,23.0,81.0,M,33.0,Non-carrier,...,3.713636,1.502427,0.342423,-1.160004,2.488551,1.628389,0.912222,0.354108,1.460335,0.569799
4,Riluzole,Rockefeller,Y,White/NH,18.0,23.0,81.0,M,33.0,Non-carrier,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.283843,1.469822,0.359835,-1.109987,1.964731,1.214844,1.082785,-0.229148,1.823705,0.722950
253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.216607,NaN,NaN
254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254956,NaN,NaN
255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.340067,2.008600,0.773786,-1.234814,2.632457,1.808886,1.143015,NaN,1.562293,0.369228


In [66]:
data_no_fdg = pd.merge(data_no_fdg, data_miscimaging_ril, how = 'outer')
data_no_fdg

,Group,Site,Study_Completed,race,Education_years,MMSE,age,sex,APOE,apoe4_status,...,Aseg_vol_R_CHORPLEX,Aseg_vol_R_HIP,Aseg_vol_R_INF_LATVENT,Aseg_vol_R_LATVENT,Aseg_vol_R_PALLIDUM,Aseg_vol_R_PUTAMEN,Aseg_vol_R_THALAMUS,Aseg_vol_R_VENTRALDC,Aseg_vol_R_VESSEL,Aseg_vol_B_WM_HYPOINT
0,Placebo,Rockefeller,Y,White/NH,14.0,26.0,76.0,M,23.0,Non-carrier,...,540.6,3432.0,177.4,13629.8,1787.3,3335.8,5590.7,3232.2,56.1,2522.7
1,Placebo,Rockefeller,Y,White/NH,14.0,26.0,76.0,M,23.0,Non-carrier,...,613.1,3341.3,216.2,14527.6,1927.6,3496.8,5631.8,3077.9,85.1,1994.7
2,Placebo,Rockefeller,Y,White/NH,14.0,26.0,76.0,M,23.0,Non-carrier,...,656.8,3252.9,277.7,13762.8,1863.5,3155.3,5452.4,3006.3,71.7,2563.2
3,Riluzole,Rockefeller,Y,White/NH,18.0,23.0,81.0,M,33.0,Non-carrier,...,1043.2,3796.4,1580.5,32538.3,2436.2,5944.4,6849.1,3295.2,113.1,9032.4
4,Riluzole,Rockefeller,Y,White/NH,18.0,23.0,81.0,M,33.0,Non-carrier,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,754.4,3793.1,1009.9,18990.2,2156.4,3856.3,6330.0,3847.2,11.0,3077.9
258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,561.4,3488.3,645.6,14426.4,1686.7,4269.8,6402.2,3183.5,8.9,9157.0


In [67]:
# define groups
data_no_fdg['source'] = data_no_fdg['Diagnosis'].copy()
data_no_fdg.loc[data_no_fdg['Subject_Label'].isin(['APE-792', 'RIL']), 'source'] = 'Riluzole Trial Participants' 
data_no_fdg.loc[data_no_fdg['Subject_Label'].isin(['APE-767']), 'source'] = 'Control' 
data_no_fdg['source'] = data_no_fdg['source'].replace({'AD':'Non-Trial AD', 'MCI':'Non-Trial MCI', 'Other':'Non-Trial Other Dx', np.nan:'Non-Trial Unknown Dx'})


data_no_fdg['Group2'] = data_no_fdg['Group'].copy()
data_no_fdg.loc[data_no_fdg['Group'].isin(['Placebo', 'Riluzole']), 'Group2'] = 'Riluzole Trial Participants'

data_no_fdg['Group3'] = data_no_fdg['Group'].copy()
data_no_fdg.loc[data_no_fdg['Subject_Label'].isin(['AANDS', 'ADRC']) & data_no_fdg['Group'].isin(['Control']), 'Group3'] = 'ADRC Control'
data_no_fdg.loc[data_no_fdg['Subject_Label'].isin(['APE-767']) & data_no_fdg['Group'].isin(['Control']), 'Group3'] = 'Sleep Study Control'
data_no_fdg.loc[data_no_fdg['Group3'].isin(['Control']), 'Group3'] = np.nan

# rename regions
data_no_fdg.columns = data_no_fdg.columns.str.replace(' |-', '_', regex = True)

# order
data_no_fdg = data_no_fdg.sort_values(['timepoint', "ID_ADM"])

# Combine non FDG with FDG

In [68]:
data_FDG_rr_orig_merge = pd.merge(data_FDG_rr_orig, data_no_fdg, how = 'outer')
data_FDG_rr_orig_merge.columns = data_FDG_rr_orig_merge.columns.str.replace(' |-', '_', regex = True)

data_FDG_rr_pons_merge = pd.merge(data_FDG_rr_pons, data_no_fdg, how = 'outer')
data_FDG_rr_pons_merge.columns = data_FDG_rr_pons_merge.columns.str.replace(' |-', '_', regex = True)

data_FDG_rr_para_merge = pd.merge(data_FDG_rr_para, data_no_fdg, how = 'outer')
data_FDG_rr_para_merge.columns = data_FDG_rr_para_merge.columns.str.replace(' |-', '_', regex = True)


data_FDG_rr_para_new_merge = pd.merge(data_FDG_rr_para_new, data_no_fdg, how = 'outer')
data_FDG_rr_para_new_merge.columns = data_FDG_rr_para_new_merge.columns.str.replace(' |-', '_', regex = True)

data_FDG_rr_ras_new_merge = pd.merge(data_FDG_rr_ras_new, data_no_fdg, how = 'outer')
data_FDG_rr_ras_new_merge.columns = data_FDG_rr_ras_new_merge.columns.str.replace(' |-', '_', regex = True)


In [69]:
# get list of unique regions
list(set(data_FDG_rr_orig.drop(columns = ['Subject_ID', 'Subject_Label', 'Subject_Number', 'timepoint', 'ID_ADM']
).columns.str.replace(' |-', '_', regex = True).to_list() + data_FDG_rr_pons.drop(columns = ['Subject_ID', 'Subject_Label', 'Subject_Number', 'timepoint', 'ID_ADM']
).columns.str.replace(' |-', '_', regex = True).to_list() + data_FDG_rr_para.drop(columns = ['Subject_ID', 'Subject_Label', 'Subject_Number', 'timepoint', 'ID_ADM']
).columns.str.replace(' |-', '_', regex = True).to_list() + data_FDG_rr_para_new.drop(columns = ['Subject_ID', 'Subject_Label', 'Subject_Number', 'timepoint', 'ID_ADM']
).columns.str.replace(' |-', '_', regex = True).to_list() + data_FDG_rr_ras_new.drop(columns = ['Subject_ID', 'Subject_Label', 'Subject_Number', 'timepoint', 'ID_ADM']
).columns.str.replace(' |-', '_', regex = True).to_list()))

['SensMot_gm',
 'R_Hip',
 'Temp_gm',
 'CO',
 'Graycer_gm',
 'Par_gm',
 'L_Hip',
 'Avg_PCC',
 'Temp',
 'MTL_gm',
 'AC_gm',
 'Avg_MedOrbFrontal',
 'FRONTAL_gm',
 'Paracentral_gm',
 'PostCing_gm',
 'Precun_gm',
 'Avg_Hip',
 'RASref1_gm',
 'Vermis_gm']

# Remove Excluded (non-AD) subjects

In [70]:
# list of the 11 new subjects
data_FDG_rr_ras_new_merge.loc[~data_FDG_rr_ras_new_merge['ID_ADM'].isin(data_FDG_rr_orig_merge.loc[data_FDG_rr_orig_merge['Temp_gm'].notna() ,'ID_ADM']) &  data_FDG_rr_ras_new_merge['Temp_gm'].notna(), 'ID_ADM'].unique()

array(['APE-792_23', 'APE-792_35', 'APE-792_39', 'APE-792_65',
       'APE-792_71', 'RIL_17', 'RIL_21', 'RIL_24', 'APE-792_9',
       'APE-792_22', 'RIL_13'], dtype=object)

In [71]:
data_FDG_rr_ras_new_merge['ID_ADM'].sort_values().unique()

array(['AANDS_10', 'AANDS_105', 'AANDS_107', 'AANDS_11', 'AANDS_113',
       'AANDS_116', 'AANDS_28', 'AANDS_3', 'AANDS_33', 'AANDS_35',
       'AANDS_41', 'AANDS_42', 'AANDS_45', 'AANDS_46', 'AANDS_47',
       'AANDS_49', 'AANDS_51', 'AANDS_62', 'AANDS_63', 'AANDS_64',
       'AANDS_78', 'AANDS_89', 'ADRC_11068', 'ADRC_147082', 'ADRC_2800',
       'ADRC_48043', 'ADRC_52464', 'ADRC_55745', 'ADRC_63118',
       'ADRC_80257', 'ADRC_93371', 'APE-767_10', 'APE-767_11',
       'APE-767_12', 'APE-767_13', 'APE-767_14', 'APE-767_15',
       'APE-767_16', 'APE-767_17', 'APE-767_18', 'APE-767_19',
       'APE-767_2', 'APE-767_20', 'APE-767_21', 'APE-767_23',
       'APE-767_24', 'APE-767_25', 'APE-767_26', 'APE-767_28',
       'APE-767_3', 'APE-767_30', 'APE-767_31', 'APE-767_32',
       'APE-767_33', 'APE-767_34', 'APE-767_36', 'APE-767_37',
       'APE-767_38', 'APE-767_39', 'APE-767_4', 'APE-767_40',
       'APE-767_41', 'APE-767_42', 'APE-767_43', 'APE-767_44',
       'APE-767_45', 'APE-767

In [72]:
excluded_subjects = ['APE-792_39', 'APE-792_23', 'APE-792_35', 'RIL_21', 'RIL_8']

data_FDG_rr_orig_merge = data_FDG_rr_orig_merge[~data_FDG_rr_orig_merge['ID_ADM'].isin(excluded_subjects)]
data_FDG_rr_pons_merge = data_FDG_rr_pons_merge[~data_FDG_rr_pons_merge['ID_ADM'].isin(excluded_subjects)]
data_FDG_rr_para_merge = data_FDG_rr_para_merge[~data_FDG_rr_para_merge['ID_ADM'].isin(excluded_subjects)]
data_FDG_rr_para_new_merge = data_FDG_rr_para_new_merge[~data_FDG_rr_para_new_merge['ID_ADM'].isin(excluded_subjects)]
data_FDG_rr_ras_new_merge = data_FDG_rr_ras_new_merge[~data_FDG_rr_ras_new_merge['ID_ADM'].isin(excluded_subjects)]

In [73]:
data_FDG_rr_ras_new_merge.loc[~data_FDG_rr_ras_new_merge['ID_ADM'].isin(data_FDG_rr_orig_merge.loc[data_FDG_rr_orig_merge['Temp_gm'].notna() ,'ID_ADM']) &  data_FDG_rr_ras_new_merge['Temp_gm'].notna(), 'ID_ADM'].unique()

array(['APE-792_65', 'APE-792_71', 'RIL_17', 'RIL_24', 'APE-792_9',
       'APE-792_22', 'RIL_13'], dtype=object)

In [74]:
data_FDG_rr_ras_new_merge.loc[ data_FDG_rr_ras_new_merge['DKT_vol_L_CUNEUS'].isna() & data_FDG_rr_ras_new_merge['source'].isin(['Riluzole Trial Participants']) & data_FDG_rr_ras_new_merge['timepoint'].isin(['base']), 'ID_ADM'].unique()

array(['APE-792_26', 'RIL_1', 'RIL_27', 'RIL_2', 'RIL_3', 'RIL_9',
       'APE-792_47', 'APE-792_56', 'APE-792_66', 'RIL_31', 'APE-792_55',
       'APE-792_60', 'RIL_5'], dtype=object)

In [75]:
data_FDG_rr_para_new_merge[data_FDG_rr_para_new_merge['Subject_ID'].str.contains('62')==True][['Subject_Label', 'Subject_Number', 'ID_ADM', 'timepoint', 'pTau217_log10']]

,Subject_Label,Subject_Number,ID_ADM,timepoint,pTau217_log10
39,APE-792,62,APE-792_62,base,-0.572711


In [76]:
data_FDG_rr_para_new_merge[data_FDG_rr_para_new_merge['ID_ADM'].str.contains('8')==True][['Subject_Label', 'Subject_Number', 'ID_ADM', 'timepoint', 'pTau217', 'MTL_gm', ]]

,Subject_Label,Subject_Number,ID_ADM,timepoint,pTau217,MTL_gm
0,APE-792,18,APE-792_18,base,NaN,0.619338
3,RIL,18,RIL_18,base,NaN,0.693443
7,RIL,28,RIL_28,base,1.150000,0.639254
13,APE-792,58,APE-792_58,base,NaN,0.669421
61,APE-792,18,APE-792_18,end,NaN,NaN
74,APE-792,58,APE-792_58,end,NaN,NaN
84,RIL,18,RIL_18,end,1.790000,NaN
91,RIL,28,RIL_28,end,1.140000,NaN
103,APE-792,18,APE-792_18,mid,0.920000,NaN
116,APE-792,58,APE-792_58,mid,1.251808,NaN


# Save

In [77]:
import pickle
# Saving the objects:
with open(code_folder_loc + '/output/01_newdata.pkl', 'wb') as f: 
    pickle.dump([data_FDG_rr_orig_merge, data_FDG_rr_pons_merge, data_FDG_rr_para_merge, data_FDG_rr_para_new_merge, data_FDG_rr_ras_new_merge], f)

In [78]:
# save raw dataset as CSV
temp_df = data_FDG_rr_para_new_merge.copy()
temp_df = temp_df[temp_df['Group2'].isin(['Riluzole Trial Participants'])].dropna(axis = 1, how = 'all')

column_order = ['ID_ADM', 'Subject_ID',  'Subject_Label', 'Subject_Number', 'timepoint',
 'source', 'Group','Group2', 'Group3',  'Diagnosis', 'Site', 'Study_Completed', 
      
      'race', 'Education_years',  'age', 'age_decade', 'age_decade2', 

       'sex', 'APOE', 'apoe4_status', 'apoe4_carrier', 

       
       'pTau217', 'pTau231', 'Ab40', 'Ab42',
       'GFAP', 'NFL', 'pTau181', 'Ab42_40', 'pTau181_Ab42', 'Ab40_log10',
       'Ab42_log10', 'Ab42_40_log10', 'GFAP_log10', 'NFL_log10',
       'pTau181_log10', 'pTau217_log10', 'pTau231_log10', 'pTau181_Ab42_log10',


       'Temp_gm', 'MTL_gm', 'PostCing_gm', 'Par_gm', 'Precun_gm',
       'SensMot_gm', 'Paracentral_gm', 'AC_gm', 'RASref1_gm', 'FRONTAL_gm',
       
       'MMSE', 'adascogtotal',
       'adaswrm', 'adasn', 'adasc', 'adascp', 'adasip', 'adaso', 'adaswr',
       'adasl', 'adascsl', 'adaswfd', 'adasrti', 'adasdr', 'bvrt', 'dstotal',
       'dsf', 'dsb', 'ldsf', 'ldsb', 'tma', 'lines_completeda', 'tmae', 'tmb',
       'lines_completedb', 'tmbe', 'cowatf_c_f', 'cowata_f_a', 'cowats_l_s',
       'cowattotal', 'animals_SI', 'clockdraw', 'clockcopy', 'cdrtotal',
       'cdrsum', 'cdrmem', 'cdro', 'cdrjps', 'cdrca', 'cdrhh', 'cdrpc',
       'adltotal', 'npid', 'npih', 'npiaa', 'npidd', 'npia', 'npiee', 'npiai',
       'npidisinh', 'npiil', 'npimd', 'npinb', 'npiae', 'npitotal',
       'npicaregdis', 'npidomains', 'lm1', 'lm2', 'lm2recog', 'gds',
       'naart_incorrect', 'NAART_VIQ',
       
   
       'ImageNames', 'VOL_Ventricles_Lz', 'VOL_Ventricles_Rz',
       'VOL_Putamen_Lz', 'VOL_Putamen_Rz', 'VOL_ParaHip_Lz', 'VOL_ParaHip_Rz',
       'VOL_Fusi_Lz', 'VOL_Fusi_Rz', 'VOL_InfTemp_Lz', 'VOL_InfTemp_Rz',
       'VOL_MidTemp_Lz', 'VOL_MidTemp_Rz', 'VOL_SupTemp_Lz', 'VOL_SupTemp_Rz',
       'VOL_Precun_Lz', 'VOL_Precun_Rz', 'VOL_InfPar_Lz', 'VOL_InfPar_Rz',
       'VOL_ParaPostCentr_Lz', 'VOL_ParaPostCentr_Rz', 'VOL_SupraMarg_Lz',
       'VOL_SupraMarg_Rz', 'VOL_SupPar_Lz', 'VOL_SupPar_Rz',
       'VOL_OrbitFront_Lz', 'VOL_OrbitFront_Rz', 'VOL_Insula_Lz',
       'VOL_Insula_Rz', 'VOL_InfFront_Lz', 'VOL_InfFront_Rz',
       'VOL_MidFront_Lz', 'VOL_MidFront_Rz', 'VOL_SupFront_Lz',
       'VOL_SupFront_Rz', 'VOL_PrecFront_Lz', 'VOL_PrecFront_Rz',
       'VOL_LatOcc_Lz', 'VOL_LatOcc_Rz', 'VOL_Lingual_Lz', 'VOL_Lingual_Rz',
       'VOL_Cuneus_Lz', 'VOL_Cuneus_Rz', 'VOL_Pericalc_Lz', 'VOL_Pericalc_Rz',
       'VOL_AntCingulate_Lz', 'VOL_AntCingulate_Rz', 'VOL_PostCingulate_Lz',
       'VOL_PostCingulate_Rz', 'VOL_Entorhinal_Lz', 'VOL_Entorhinal_Rz',
       'VOL_Hip_Lz', 'VOL_Hip_Rz', 'VOL_TotalGrayz', 'VOL_TotalGray_Lz',
       'VOL_TotalGray_Rz', 'VOL_LatTemp_Lz', 'VOL_LatTemp_Rz',
       'VOL_Parietal_Lz', 'VOL_Parietal_Rz', 'VOL_Frontal_Lz',
       'VOL_Frontal_Rz', 'VOL_InfMidTemp_Lz', 'VOL_InfMidTemp_Rz',
       'VOL_InfInsFrontal_Lz', 'VOL_InfInsFrontal_Rz', 'VOL_MidSupFrontal_Lz', 'VOL_MidSupFrontal_Rz',
       'VOL_Inf_Mid_Fus_Temp_Lz', 'VOL_Inf_Mid_Fus_Temp_Rz',
       'VOL_Precun_InfPar_Lz', 'VOL_Precun_InfPar_Rz',
       'VOL_Precun_InfPar_Supramarg_Lz', 'VOL_Precun_InfPar_Supramarg_Rz',
       'VOL_LatOccLingCun_Lz', 'VOL_LatOccLingCun_Rz', 'VOL_InfParSupra_Lz',
       'VOL_InfParSupra_Rz', 'MRI_Visit_Date', '7_Character_machine_UID',
       'MRI_ReName', 'SPM12_Source_Mat_file', 'GM', 'WM', 'CSF', 'Total',
       'FS_COMMENT(s)', 'Medulla', 'Pons.5', 'SCP', 'Midbrain',
       'Whole_brainstem', 'L_Hippocampal_tail', 'L_subiculum', 'L_CA1',
       'L_hippocampal_fissure', 'L_presubiculum', 'L_parasubiculum',
       'L_molecular_layer_HP', 'L_GC_ML_DG', 'L_CA3', 'L_CA4', 'L_fimbria',
       'L_HATA', 'L_Whole_hippocampus', 'R_Hippocampal_tail', 'R_subiculum',
       'R_CA1', 'R_hippocampal_fissure', 'R_presubiculum', 'R_parasubiculum',
       'R_molecular_layer_HP', 'R_GC_ML_DG', 'R_CA3', 'R_CA4', 'R_fimbria',
       'R_HATA', 'R_Whole_hippocampus', 'DKT_vol_L_CING_CAUDANT',
       'DKT_vol_L_CAUDMIDFRN', 'DKT_vol_L_CUNEUS', 'DKT_vol_L_ENTORHINAL',
       'DKT_vol_L_FUSI', 'DKT_vol_L_INFPAR', 'DKT_vol_L_INFTEMP',
       'DKT_vol_L_INS', 'DKT_vol_L_CING_ISTHMUS', 'DKT_vol_L_LATOCC',
       'DKT_vol_L_LATORBFRN', 'DKT_vol_L_LATLING', 'DKT_vol_L_LATMEDORBFRN',
       'DKT_vol_L_LATMIDTEMP', 'DKT_vol_L_PARACNTRL', 'DKT_vol_L_PHG',
       'DKT_vol_L_PARSOPERC', 'DKT_vol_L_PARSORBIT', 'DKT_vol_L_PARSTRIANG',
       'DKT_vol_L_PERICALC', 'DKT_vol_L_POSTCNTRL', 'DKT_vol_L_CING_POST',
       'DKT_vol_L_PRECNTRL', 'DKT_vol_L_PRECUNEUS', 'DKT_vol_L_CING_ROSTANT',
       'DKT_vol_L_ROSTMIDFRN', 'DKT_vol_L_SUP_FRONT', 'DKT_vol_L_SUP_PAR',
       'DKT_vol_L_SUP_TEMP', 'DKT_vol_L_SUPRAMARG', 'DKT_vol_L_TRANSVTEMP',
       'DKT_vol_R_CING_CAUDANT', 'DKT_vol_R_CAUDMIDFRN', 'DKT_vol_R_CUNEUS',
       'DKT_vol_R_ENTORHINAL', 'DKT_vol_R_FUSI', 'DKT_vol_R_INFPAR',
       'DKT_vol_R_INFTEMP', 'DKT_vol_R_INS', 'DKT_vol_R_CING_ISTHMUS',
       'DKT_vol_R_LATOCC', 'DKT_vol_R_LATORBFRN', 'DKT_vol_R_LATLING',
       'DKT_vol_R_LATMEDORBFRN', 'DKT_vol_R_LATMIDTEMP', 'DKT_vol_R_PARACNTRL',
       'DKT_vol_R_PHG', 'DKT_vol_R_PARSOPERC', 'DKT_vol_R_PARSORBIT', 'DKT_vol_R_PARSTRIANG',
       'DKT_vol_R_PERICALC', 'DKT_vol_R_POSTCNTRL', 'DKT_vol_R_CING_POST',
       'DKT_vol_R_PRECNTRL', 'DKT_vol_R_PRECUNEUS', 'DKT_vol_R_CING_ROSTANT',
       'DKT_vol_R_ROSTMIDFRN', 'DKT_vol_R_SUP_FRONT', 'DKT_vol_R_SUP_PAR',
       'DKT_vol_R_SUP_TEMP', 'DKT_vol_R_SUPRAMARG', 'DKT_vol_R_TRANSVTEMP',
       'AparcSt_vol_L_SSTSBANK', 'AparcSt_vol_L_FRNPOLE',
       'AparcSt_vol_L_TEMPPOLE', 'AparcSt_vol_R_SSTSBANK',
       'AparcSt_vol_R_FRNPOLE', 'AparcSt_vol_R_TEMPPOLE', 'Aseg_vol_B_3RDVENT',
       'Aseg_vol_B_4THVENT', 'Aseg_vol_B_BSTEM', 'Aseg_vol_B_CC_ANT',
       'Aseg_vol_B_CC_CENT', 'Aseg_vol_B_CC_MIDANT', 'Aseg_vol_B_CC_MIDPOST',
       'Aseg_vol_B_CC_POST', 'Aseg_vol_B_CSF', 'Aseg_vol_L_ACCUMBENS',
       'Aseg_vol_L_AMYGDALA', 'Aseg_vol_L_CAUD', 'Aseg_vol_L_CTX_CBL',
       'Aseg_vol_L_WM_CBL', 'Aseg_vol_L_CHORPLEX', 'Aseg_vol_L_HIP',
       'Aseg_vol_L_INFLATVENT', 'Aseg_vol_L_LATVENT', 'Aseg_vol_L_PALLIDUM',
       'Aseg_vol_L_PUTAMEN', 'Aseg_vol_L_THALAMUS', 'Aseg_vol_L_VENTRALDC',
       'Aseg_vol_L_VESSEL', 'Aseg_vol_B_OPTICCHIASM', 'Aseg_vol_R_ACCUMBENS',
       'Aseg_vol_R_AMYGDALA', 'Aseg_vol_R_CAUD', 'Aseg_vol_R_CTX_CBL',
       'Aseg_vol_R_WM_CBL', 'Aseg_vol_R_CHORPLEX', 'Aseg_vol_R_HIP',
       'Aseg_vol_R_INF_LATVENT', 'Aseg_vol_R_LATVENT', 'Aseg_vol_R_PALLIDUM',
       'Aseg_vol_R_PUTAMEN', 'Aseg_vol_R_THALAMUS', 'Aseg_vol_R_VENTRALDC',
       'Aseg_vol_R_VESSEL', 'Aseg_vol_B_WM_HYPOINT', 


              
        'MRS_Date', 'PCC_Notes',
       'PCC_Internal_Water', 'PCC_GSH_W', 'PCC_GLU_W', 'PCC_GLX_W',
       'PCC_CHO_W', 'PCC_CR_W', 'PCC_NAA_W', 'PCC_NAA_CR', 'PCC_GLU_CR',
       'PCC_GLU_NAA',
       'LH_Notes', 'LH_Internal_Water', 'LH_GLX_W', 'LH_GABA_W', 'LH_CHO_W',
       'LH_CR_W', 'LH_NAA_W', 'RH_Notes', 'RH_Internal_Water', 'RH_GLX_W',
       'RH_GABA_W', 'RH_CHO_W', 'RH_CR_W', 'RH_NAA_W']


column_order = column_order + temp_df.columns.to_list()

column_order = list(dict.fromkeys(column_order))

temp_df = temp_df[column_order]

temp_df.to_csv(code_folder_loc + '/output/merged_dataset_new_para_rr.csv', index = False)

In [79]:
data_FDG_rr_para_new_merge[data_FDG_rr_para_new_merge['timepoint'].isin(['base']) & data_FDG_rr_para_new_merge['Temp_gm'].notna() & data_FDG_rr_para_new_merge['pTau217'].notna()][[
    'Subject_ID', 'pTau217_log10', 'pTau217',
    'age', 'sex', 'race', 'Education_years', 'apoe4_carrier', 'MMSE',
    'ID_ADM', 'Group', 'Site', 'ImageNames', 'MRI_Visit_Date', 'MRI_ReName', 'SPM12_Source_Mat_file'
    ]].to_csv(code_folder_loc + '/output/Riluzole_ptau217_forDawn.csv', index = False)

In [80]:
data_FDG_rr_para_new_merge[data_FDG_rr_para_new_merge['ID_ADM'].isin(['RIL_12']) & data_FDG_rr_para_new_merge['timepoint'].isin(['base']) & data_FDG_rr_para_new_merge['Temp_gm'].notna() & data_FDG_rr_para_new_merge['pTau217'].notna()][[
    'Subject_ID', 'pTau217_log10', 'pTau217',
    'age', 'sex', 'race', 'Education_years', 'apoe4_carrier', 'MMSE',
    'ID_ADM', 'Group', 'Site', 'ImageNames', 'MRI_Visit_Date', 'MRI_ReName', 'SPM12_Source_Mat_file'
    ]]

,Subject_ID,pTau217_log10,pTau217,age,sex,race,Education_years,apoe4_carrier,MMSE,ID_ADM,Group,Site,ImageNames,MRI_Visit_Date,MRI_ReName,SPM12_Source_Mat_file
30,RIL-012,-0.031517,0.93,82.0,F,White/NH,12.0,Yes,20.0,RIL_12,Riluzole,Sinai,RKF-RIL-012_m00_na_NuNtv.nii,2018-12-18 00:00:00,RKF-RIL-012__m00__T1-a__20181218__e013304.nii,'M:\RKF_spm12_061119\seg8-mat\RKF-RIL-012__m00...


In [81]:
temp_df = data_FDG_rr_para_new_merge.loc[data_FDG_rr_para_new_merge['ID_ADM'].isin(['RIL_12']),:]

In [82]:
data_FDG_rr_para_new_merge.loc[data_FDG_rr_para_new_merge['ID_ADM'].isin(['RIL_12']),['timepoint', 'pTau217_log10']]

,timepoint,pTau217_log10
30,base,-0.031517
81,end,0.053078
125,mid,NaN


In [83]:
len(data_FDG_rr_orig_merge.loc[data_FDG_rr_orig_merge['timepoint'].isin(['base']) & data_FDG_rr_orig_merge['Temp_gm'].notna() & data_FDG_rr_orig_merge[plasma_columns].notna().any(axis = 1), 'Subject_ID' ].unique())

38

In [84]:
 data_FDG_rr_orig_merge.loc[data_FDG_rr_orig_merge['timepoint'].isin(['base']) & data_FDG_rr_orig_merge['Temp_gm'].notna(), plasma_columns + ['ID_ADM']].notna().sum(axis = 1).value_counts().sort_index()

1      3
2      1
5      1
6      4
7      1
8      1
9      2
10    28
dtype: int64

In [85]:
data_FDG_rr_orig_merge.loc[data_FDG_rr_orig_merge['timepoint'].isin(['base']) & data_FDG_rr_orig_merge['Temp_gm'].notna(), plasma_columns].dropna(how = 'all').reset_index(drop=True)#.notna().sum()#.describe()[['min', 'max']]

,Ab40,Ab42,Ab42_40,GFAP,NFL,pTau181,pTau217,pTau231,pTau181_Ab42
0,40.1,2.94,0.073317,165.0,69.3,7.23,NaN,NaN,2.459184
1,67.1,3.98,0.059314,271.0,27.4,10.50,0.930000,39.956930,2.638191
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.970815,NaN
3,47.4,3.48,0.073418,92.0,23.2,3.80,NaN,15.172902,1.091954
4,108.0,7.53,0.069722,164.0,47.6,6.46,2.220000,21.924868,0.857902
5,79.0,4.98,0.063038,440.0,42.0,16.90,3.610000,43.691335,3.393574
6,85.3,4.38,0.051348,239.0,37.8,11.40,2.680000,28.317818,2.602740
7,107.0,4.93,0.046075,209.0,21.7,4.07,1.150000,9.659859,0.825558
8,NaN,NaN,NaN,110.0,33.5,17.90,0.570000,61.495740,NaN
9,23.8,NaN,NaN,201.0,41.3,18.20,3.210000,40.988460,NaN


In [86]:
data_FDG_rr_pons_merge.loc[data_FDG_rr_pons_merge['timepoint'].isin(['base']) & data_FDG_rr_pons_merge['Temp_gm'].notna(), plasma_columns].notna().sum().describe()[['min', 'max']]

min    31.0
max    37.0
dtype: float64

In [87]:
len(data_FDG_rr_pons_merge.loc[data_FDG_rr_pons_merge['timepoint'].isin(['base']) & data_FDG_rr_pons_merge['Temp_gm'].notna() & data_FDG_rr_pons_merge[plasma_columns].notna().any(axis = 1), 'Subject_ID' ].unique())

38

In [88]:
data_FDG_rr_para_merge.loc[data_FDG_rr_para_merge['timepoint'].isin(['base']) & data_FDG_rr_para_merge['Temp_gm'].notna(), plasma_columns].notna().sum().describe()[['min', 'max']]

min    31.0
max    37.0
dtype: float64

In [89]:
len(data_FDG_rr_para_merge.loc[data_FDG_rr_para_merge['timepoint'].isin(['base']) & data_FDG_rr_para_merge['Temp_gm'].notna() & data_FDG_rr_para_merge[plasma_columns].notna().any(axis = 1), 'Subject_ID' ].unique())

38

In [90]:
data_FDG_rr_para_new_merge.loc[data_FDG_rr_para_new_merge['timepoint'].isin(['base']) & data_FDG_rr_para_new_merge['Temp_gm'].notna(), plasma_columns].notna().sum().describe()[['min', 'max']]

min    36.0
max    42.0
dtype: float64

In [91]:
len(data_FDG_rr_para_new_merge.loc[data_FDG_rr_para_new_merge['timepoint'].isin(['base']) & data_FDG_rr_para_new_merge['Temp_gm'].notna() & data_FDG_rr_para_new_merge[plasma_columns].notna().any(axis = 1), 'Subject_ID' ].unique())

43

In [92]:
data_FDG_rr_ras_new_merge.loc[data_FDG_rr_ras_new_merge['timepoint'].isin(['base']) & data_FDG_rr_ras_new_merge['Temp_gm'].notna(), plasma_columns].notna().sum().describe()[['min', 'max']]

min    36.0
max    42.0
dtype: float64

In [93]:
len(data_FDG_rr_ras_new_merge.loc[data_FDG_rr_ras_new_merge['timepoint'].isin(['base']) & data_FDG_rr_ras_new_merge['Temp_gm'].notna() & data_FDG_rr_ras_new_merge[plasma_columns].notna().any(axis = 1), 'Subject_ID' ].unique())

43